# DA202 · Customer Segmentation: Mastery Project
* Author: Alan Kong @[LinkedIn](https://www.linkedin.com/in/alan-kong-professional/)
* Dashboard @[Tableau Public](https://public.tableau.com/views/5_Dashboard_TravelTide_AKclean/Dash?:language=en-US&:display_count=n&:origin=viz_share_link)
* To replicate the analysis [data.csv](https://github.com/coderedstorage/TravelTide/commit/e858a92d61a1c334054415c7c8b1edb88e48f7e7) is available on GitHub. Please skip Steps B-D in the python notebook as required information to run those sections are too large and not provided on GitHub

# A. Preparation

#### 1. Import libraries

In [1]:
import numpy as np
import warnings
import math
import pandas as pd
import matplotlib.pyplot as plt
import os
import psycopg2
import ipywidgets as widgets
import time
import random
from IPython.display import display, clear_output
from geopy.distance import geodesic
from geopy.distance import great_circle
from geographiclib.geodesic import Geodesic
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from sklearn.cluster import KMeans
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from pandas.tseries.offsets import MonthBegin

#### 2. Pre-set paths and check the directories exist and write-able

In [2]:
BASE_PATH = r"C:\\data\\tt\\py\\"                   # preset destinations for output

CONFIG_PATH = {k: f"{BASE_PATH}{v}" for k, v in {
    "full": "full.csv", "data": "data.csv", 
    "precut": "precut.csv", "clean": "clean.csv", "outliers": "outliers.csv", # transaction level data
    "user": "user.csv", "selection": "selection.csv", # user level data
    "corr_matrix": "corr_matrix.csv", "sample_size_matrix": "sample_size_matrix.csv", # user level data
    "kmeans_t": "kmeans_t.csv", "kmeans_u": "kmeans_u.csv",                 # kmeans
    "dummy": "dummy.csv" # dummy
}.items()}
CONFIG_PATH["chromedriver.exe"] = r"C:\\scrapper\\chromedriver.exe"

def check_directories():                            # function to check preset destinations for output
    all_good = True  # Initialize flag for directory status
    for key, path in CONFIG_PATH.items():
        directory = os.path.dirname(path)
        msg = f"Problem with {key} [{path}]\n"
        
        if not os.path.exists(directory) or not os.access(directory, os.W_OK):
            all_good = False  # Set flag to False
            if not os.path.exists(directory):
                msg += "• Directory doesn't exist. Creating now...\n"
                os.makedirs(directory)
            else:
                msg += "• Directory exists but has no write permissions\n"
            print(msg)  # Print the message and adds an empty line
    if all_good:
        print("All directories exist with write permissions")
    return all_good

all_good = check_directories() # call the function
# All directories exist with write permissions

All directories exist with write permissions


#### 3. Practical code snippets for adhoc use

In [3]:
dummy = pd.DataFrame({                                                      # dummy df to test some of the snippets
    'id': ['r1', 'r2', 'r3', 'r4', 'r5', 'r6'],
    'col1': [100, 200, 150, np.nan, 180, 220],
    'col2': [0.5, 0.2, 0.1, np.nan, 0.25, 0.15],
    'col3': ['Hilton', 'InterContinental', np.nan, 'Mariott', 'Banyan Tree', 'Ramada'],
    'col4': ['Houston', 'SF', 'SF', np.nan, 'Miami', 'LA'],
    'lat1': [f"{lat:.4f}" for lat in [40.7128, 34.0522, 50.7128, 45.0522, 40.7128, 35.0522]],
    'lon1': [f"{lat:.4f}" for lat in [50.7128, 44.0522, 60.7128, 54.0522, 60.7128, 44.0522]],
    'lat2': [f"{lat:.4f}" for lat in [35.7128, 25.0522, 5.7128, 35.0522, 44.7128, 55.0522]],
    'lon2': [f"{lat:.4f}" for lat in [25.7128, 30.0522, 35.7128, 45.0522, 50.7128, 58.0522]],    
    'cancellation': [False, False, True, np.nan, False, True],
    'flight_booked': [True, False, True, np.nan, False, True],
    'return_flight_booked': [True, False, np.nan, np.nan, False, False],
    'clicks': ['12.0 ', ' 555', '568.0', ' 65.0  ', ' 7.0 ', ' 5.0']})
pd.DataFrame(list(dummy.columns), columns=['variables'])                    # convert a list to df

,variables
0,id
1,col1
2,col2
3,col3
4,col4
5,lat1
6,lon1
7,lat2
8,lon2
9,cancellation


* Call without assigning df, automatically modifies original df in arg: def(arg)

In [4]:
def df_name_and_variable_sizes(df, col_list):                                           # output name of df and count unique values of its variables
    def find_df_name(df):                                                               # find df name
        name = [x for x in globals() if globals()[x] is df][0]
        print(f"df name is {name} ", end="")                                            # end="" arg so next print is in same line
    find_df_name(df)                                                                    # print df name
    unique_counts = {col: df[col].nunique() for col in col_list}
    print("==> "+ " , ".join(f"{unique_counts[col]} {col}" for col in col_list))        # print counts
df_name_and_variable_sizes(dummy, ['col1', 'col2', 'col3'])                                    # call the function

df name is dummy ==> 5 col1 , 5 col2 , 5 col3


In [6]:
# calculate flown distances, 1-way flight equivalent distance needs to be separately calculated per choice of flown distances
# lat1_col='home_airport_lat', lon1_col='home_airport_lon', lat2_col='destination_airport_lat', lon2_col='destination_airport_lon', flight_booked_col='flight_booked', return_flight_col='return_flight_booked', chosen_method='karney'
# method for flown_km and distance_km for metrics are 'haversine_basic', 'haversine_oblate', 'karney'
def distances(df, lat1_col='home_airport_lat', lon1_col='home_airport_lon', lat2_col='destination_airport_lat', lon2_col='destination_airport_lon',
              flight_booked_col='flight_booked', return_flight_col='return_flight_booked', method='karney', 
              short_haul_max=1287, medium_haul_max=4023, long_haul_max=10460):
    """
    * Haversine: Great-circle distance between two points on surface of a perfect sphere; (oblate==True) to adjust for Earth's oblateness (use weighted average of equatorial and polar radii based on latitudes of two points) 
    * Charles Karney's Method: Use geographiclib library. Geodesic.WGS84.Inverse method from geographiclib calculates distance between two points based on WGS-84 ellipsoidal model of the Earth. Recent and more accurate method for geodetic computations than adjusted haversine. Accurate results for nearly antipodal points. superior to previously commonly used Vincenty's method that might not converge
    """    
    valid_methods = ['haversine_basic', 'haversine_oblate', 'karney']
    if method not in valid_methods and method is not None:
        print(f"Invalid method chosen: {method}. Choose from {', '.join(valid_methods)}.")
        return

    def is_true(val):                                                                                                   # check if a value is true-like
        return val in [True, 'True', 'true', 1]
    
    def is_false_or_nan(val):                                                                                           # check if a value is false or NaN
        return val in [False, 'False', 'false', 0, 'NaN', 'nan', np.nan]
    
    for col in [lat1_col, lon1_col, lat2_col, lon2_col]:                                                                # ensure columns are of float type
        df[col] = df[col].astype(float)
    
    def haversine(lat1, lon1, lat2, lon2, oblate=False):                                                                # haversine estimate distances
        lat1, lon1, lat2, lon2 = map(np.radians, map(float, [lat1, lon1, lat2, lon2]))
        R = 6371 if not oblate else (1 - 1/298.257223563) * 6378.137 / np.sqrt(1 - (1/298.257223563)**2 * np.sin((lat1 + lat2) / 2)**2)
        dLat, dLon = lat2 - lat1, lon2 - lon1
        a = np.sin(dLat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dLon / 2)**2
        return R * 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)) 
    
    def categorize(distance):
        if math.isnan(distance) or distance == 0:
            return None, None
        flight_category = ('short haul' if distance <= short_haul_max else
                           'medium haul' if distance <= medium_haul_max else 
                           'long haul' if distance <= long_haul_max else 'ultra long haul')
        flight_long_haul = distance > medium_haul_max
        return flight_category, flight_long_haul
    
    def calculate_all(row):                                                                                             # main function
        if is_false_or_nan(row[flight_booked_col]):                                                                     # not accounting for cancellation, such that theoretical flight distances still calculated
            return [np.nan]*8
        lat1, lon1, lat2, lon2 = row[lat1_col], row[lon1_col], row[lat2_col], row[lon2_col]
        h_basic, h_oblate = haversine(lat1, lon1, lat2, lon2), haversine(lat1, lon1, lat2, lon2, oblate=True)           # call the function for haversine both without and with oblate adjustment
        dist_karney = Geodesic.WGS84.Inverse(lat1, lon1, lat2, lon2)['s12'] / 1000                                      # charles karney's method, conver to km
        multiplier = 2 if (row[return_flight_col]==True) else 1                                                                 # multiplier for return flight
    
        distance_km = {'haversine_basic': h_basic, 'haversine_oblate': h_oblate, 'karney': dist_karney}.get(method, 0)  # method is drawn from the keys of dict
        flown_km = distance_km * multiplier

        flight_category, flight_long_haul = categorize(distance_km)
        return [h_basic, h_oblate, dist_karney, method, flown_km, distance_km, flight_category, flight_long_haul]    
    
    
    new_cols = pd.DataFrame(df.apply(calculate_all, axis=1).tolist(),                                                   # apply main function
                            columns=['flown_km_haversine_basic', 'flown_km_haversine_oblate', 'flown_km_karney', 'distance_method', 'flown_km', 'distance_km', 'flight_category', 'flight_long_haul'])
    
    for col in new_cols.columns:
        df[col] = new_cols[col]
        
    return df

distances(dummy, lat1_col='lat1', lon1_col='lon1', lat2_col='lat2', lon2_col='lon2', 
              flight_booked_col='flight_booked', return_flight_col='return_flight_booked', method='karney', 
              short_haul_max=1287, medium_haul_max=4023, long_haul_max=10460)

display(dummy[['flight_booked', 'return_flight_booked', 'distance_method', 'flown_km', 'distance_km', 'flight_category', 'flight_long_haul']])

,flight_booked,return_flight_booked,distance_method,flown_km,distance_km,flight_category,flight_long_haul
0,True,True,karney,4500.501108,2250.250554,medium haul,False
1,False,False,NaN,NaN,NaN,NaN,NaN
2,True,NaN,karney,5503.569008,5503.569008,long haul,True
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,False,False,NaN,NaN,NaN,NaN,NaN
5,True,False,karney,2470.518028,2470.518028,medium haul,False


In [7]:
def convert_dtype(df, col_list, dtype):
    dtype_map = {
        'datetime': 'datetime64[ns]',
        'float': 'float64',
        'bool': 'bool',
        'object': 'object',
        'int': 'int64'}
    target_dtype = dtype_map.get(dtype, dtype)
    for col in col_list:
        try:
            df[col] = df[col].astype(target_dtype)                                      # motification to original df in arg made here
        except ValueError:
            if dtype == 'int':
                df[col] = df[col].astype('float64').astype(target_dtype)                # convert str to float first then int if unable to convert str to int directly

In [8]:
def convert_dtype_multi(df, dtype_dict):
    dtype_map = {
        'datetime': 'datetime64[ns]',
        'float': 'float64',
        'bool': 'bool',
        'object': 'object',
        'int': 'int64'
    }
    
    for dtype, col_list in dtype_dict.items():
        target_dtype = dtype_map.get(dtype, dtype)
        
        for col in col_list:
            try:
                df[col] = df[col].astype(target_dtype)  # In-place modification
            except ValueError:
                if dtype == 'int':
                    df[col] = df[col].astype('float64').astype(target_dtype)  # Fallback for int conversion

In [10]:
def user_cluster(df, groupby_column='user_id', 
                 n_clusters=5, random_state=42, 
                 na_as_zero=False, numeric_only=True, n_init=10):
    """
    Perform K-means clustering on a DataFrame, grouping by a specific column.
    
    Parameters:
    - df (DataFrame): Input data.
    - groupby_column (str): Column to group by.
    - n_clusters (int): Number of clusters.
    - random_state (int): Random seed for reproducibility.
    - na_as_zero (bool): Treat NaN values as zero.
    - numeric_only (bool): Whether to only consider numeric columns when aggregating.
    - n_init (int): Number of time the k-means algorithm will be run with different centroid seeds.
    
    Returns:
    - DataFrame: DataFrame with an additional 'cluster' column indicating cluster assignments.
    """
    # Create a DataFrame to store cluster labels
    cluster_df = pd.DataFrame(index=df[groupby_column].unique(), columns=['cluster'])
    
    # Handle NaN values
    if na_as_zero:
        df.fillna(0, inplace=True)
    
    # Aggregate data by the specified column, explicitly setting numeric_only
    agg_data = df.groupby(groupby_column).mean(numeric_only=numeric_only).fillna(0)
    
    # Perform K-means clustering, explicitly setting n_init
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state, n_init=n_init)
    cluster_labels = kmeans.fit_predict(agg_data)
    
    # Assign clusters
    cluster_df['cluster'] = cluster_labels
    
    # Merge the cluster labels back to the original DataFrame
    merged_df = pd.merge(df, cluster_df, how='left', left_on=groupby_column, right_index=True)
    
    return merged_df

# Example usage
# result_df = user_cluster(your_dataframe, groupby_column='user_id', na_as_zero=True)

In [12]:
def find_unmapped_values(df, mapping_dict, col_to_check):                       # find deficiency in a mapping keys to the values in a column being mapped
    all_values = set(df[col_to_check].dropna())                                 # drop NaN values
    mapped_values = set(mapping_dict.keys())
    unmapped_values = all_values - mapped_values
    
    if len(unmapped_values) > 0:
        print(f"Unmapped values in {col_to_check} column found:")
        for dest in unmapped_values:
            print(f"• {dest}")
    else:
        print(f"All values in {col_to_check} column are adequately mapped")

In [11]:
def summary(df, head_rows=5):                                                   # summary check function
    print("DATAFRAME SUMMARY".center(80, "="))
    info_df = pd.DataFrame({
        'Dtype': df.dtypes,
        'Non-Null': df.notnull().sum(),
        'Unique': df.nunique(),
        'Duplicate': df.T.duplicated().sum(),
        'NaN': df.isnull().sum()
    })
    print(info_df, f'\nRows: {df.shape[0]}', f'Columns: {df.shape[1]}')
    print("DESCRIBE".center(80, "="))
    display(df.describe())
    print("DATAFRAME HEAD".center(80, "-"))
    display(df.head(head_rows))

In [13]:
dummy.to_csv(CONFIG_PATH["dummy"], index=False)                     # export df for future use
dummy.to_csv(r"C:\\data\\tt\\py\\dummy_2.csv", index=False)         # export df for future use

In [14]:
dummy = pd.read_csv(CONFIG_PATH["dummy"])                           # import df if right version is available
dummy = pd.read_csv(r"C:\\data\\tt\\py\\dummy_2.csv")               # import df if right version is available

In [15]:
def print_quantiles(df, var_col):
    column_data = df[var_col]
    print(f"Q1 for {var_col}: {column_data.quantile(0.25)}")
    print(f"Q2 for {var_col}: {column_data.quantile(0.50)}")
    print(f"Q3 for {var_col}: {column_data.quantile(0.75)}")
# print_quantiles(dummy, 'col1')

* Call by assigning a new df name or original df in arg as function does not modify original df in arg: df = def(arg)

In [16]:
def reinsert_col(df, col_list, insert_position):                                       
    df_copy = df.copy()                                                                 # make copy of df to avoid modifying original one
    temp_df = df_copy[col_list]                                                         # store cols to move in temp df
    df_copy.drop(columns=col_list, inplace=True)                                        # drop selected cols in col_list
    for idx, col in enumerate(col_list):
        df_copy.insert(insert_position + idx, col, temp_df[col])                        # reinsert cols to desired location aka insert_position
    return df_copy  

* Call by assigning 1 new df names as fucntion does not modify original df in arg: df1 = def(arg)

In [17]:
def sample_size_matrix(df, reference_col='user_id', threshold=2):                                                                                       # sample size analysis on numeric variablesbefore and after capping at threshold multiple of stdev
    calc_stats = lambda col, mean, std: (df[reference_col][np.abs(col - mean) <= threshold * std].nunique(), col.count())                               # exclude NaN by default
    cols = [c for c in df.columns if c != reference_col] if reference_col else df.columns.tolist()
    calc_stats = lambda col, mean, std: (df[reference_col][col.notna()].nunique(), df[reference_col][np.abs(col - mean) <= threshold * std].nunique())  # (stats[0][0]), stats[0][1]) for non NaN;  (stats[1][0], stats[1][1]) for non NaN & non zero

    data = []
    for col in cols:
        if pd.api.types.is_numeric_dtype(df[col]):
            metrics, nz_metrics = df[col].agg(['count', 'mean', 'std']), df[df[col] != 0][col].agg(['count', 'mean', 'std'])
            stats = [calc_stats(df[col], metrics['mean'], metrics['std']), calc_stats(df[col], nz_metrics['mean'], nz_metrics['std'])]
            data.append([stats[0][0], stats[0][1], stats[0][1] / stats[0][0] if stats[0][0] else 0, stats[1][0], stats[1][1], stats[1][1] / stats[1][0] if stats[1][0] else 0])
    
    df_out = pd.DataFrame(data, columns=[f'all_non_NaN_values_for_{reference_col}', f'capped_non_NaN_values_within_{threshold}_std', f'ratio_non_NaN_values_capped', f'all_non_zero_values_for_{reference_col}', f'capped_non_zero_values_within_{threshold}_std', f'ratio_non_zero_values_capped'], index=cols).reset_index().rename(columns={"index": "field"}).reset_index(drop=True)
    return df_out

* Call by assigning two new df names as fucntion does not modify original df in arg: df1, df2 = def(arg)

In [18]:
def remove_outliers(df, col_list=None, col_filter='use_trip_id', reference_col='session_id', 
                    methodology='stdev', threshold=2, whisker_size=0.5):
    """
    Remove outliers based on either standard deviation or interquartile range.
    """
    # Default to all numeric columns if col_list is None
    col_list = col_list or df.select_dtypes(include=['number']).columns.tolist()
    print(f"Selected cols for {methodology}: {col_list}")
    
    # Initialize an empty list to store sets of valid row indices
    all_valid_rows = []
    
    for col in col_list:
        if pd.api.types.is_numeric_dtype(df[col]):
            if methodology == 'stdev':
                stats = df[df[col_filter]==True][col].agg(['mean', 'std'])
                valid_rows = set(df[(df[col] >= stats['mean'] - threshold * stats['std']) & 
                                    (df[col] <= stats['mean'] + threshold * stats['std'])][reference_col])
            elif methodology == 'iqr':
                Q1, Q3 = df[df[col_filter]==True][col].quantile([0.25, 0.75])
                valid_rows = set(df[(df[col] >= Q1 - whisker_size * (Q3 - Q1)) & 
                                    (df[col] <= Q3 + whisker_size * (Q3 - Q1))][reference_col])
            else:
                raise ValueError("Invalid methodology. Choose either 'stdev' or 'iqr'.")
            
            # Union with rows where the column is NaN
            nan_rows = set(df[df[col].isna()][reference_col])
            all_valid_rows.append(valid_rows.union(nan_rows))
    
    # Intersect all sets of valid rows
    valid_rows = set.intersection(*all_valid_rows)
    
    return df[df[reference_col].isin(valid_rows)], df[~df[reference_col].isin(valid_rows)]

* Call by assigning a dict and map to original df by assigning keys corresponding to variables chosen from original: minmax_dict = def(arg), df[key_minmax] = minmax_dict[key] 

In [19]:
def minmax(df, col_list=None, col_filter=None, debug=True, cap=False, threshold=2): # cap=True means to reduce the min and max range to that of threshold multiple of stdev, returns series that need to be manually added into df
    if col_list is None:
        col_list = df.select_dtypes(include=['number']).columns.tolist()
    scaled_dict = {}
    df_filtered = df[df[col_filter]] if col_filter else df                          # filter the sample based on a boolean variable if provided instead of None
    for col in col_list:
        if not pd.api.types.is_numeric_dtype(df[col]):                              # restrict to numeric variables
            warnings.warn(f"The column {col} is not numeric. Skipping.")
            continue
        stats = df_filtered[col].agg(['min', 'max', 'mean', 'std'])                 # sample statistics, most relevant if capping applied, else for reference only
        min_val, max_val = stats['min'], stats['max']                               # unadulterated min and max values for variable
        if cap:
            min_val, max_val = max(stats['mean'] - threshold * stats['std'], min_val), min(stats['mean'] + threshold * stats['std'], max_val) # capping applied if cap=True
        if debug:
            print(f"Scaling column {col}:")
            print(f"- Initial: Actual Min={stats['min']:.2f}, Max={stats['max']:.2f}. Capped Min={min_val:.2f}, Max={max_val:.2f}. Mean={stats['mean']:.2f}, Std={stats['std']:.2f}")
        scaled_series = pd.Series([0]*len(df)) if min_val == max_val else ((df[col] - min_val) / (max_val - min_val)).clip(0, 1)
        scaled_dict[col] = scaled_series                                            # series in the same order as the rows picked up from df, so can be manually added to the df later by assigning 
        if debug:
            print(f"- Scaled: Final Min={scaled_series.min():.2f}, Max={scaled_series.max():.2f}. Mean={scaled_series.mean():.2f}, Std={scaled_series.std():.2f}\n")
    return scaled_dict

In [20]:
# def with minmax embebded, does not modify the original df, need to call dummy_df = def(arg)
def compute_index_scores(df=None, index_weight_dict=None, selected_index=None, col_filter=None, debug=True, cap=True, threshold=2):
    def minmax(df, col_list=None, col_filter=None, debug=True, cap=False, threshold=2): # cap=True means to reduce the min and max range to that of threshold multiple of stdev, returns series that need to be manually added into df
        if col_list is None:
            col_list = df.select_dtypes(include=['number']).columns.tolist()
        scaled_dict = {}
        df_filtered = df[df[col_filter]] if col_filter else df                          # filter the sample based on a boolean variable if provided instead of None
        for col in col_list:
            if not pd.api.types.is_numeric_dtype(df[col]):                              # restrict to numeric variables
                warnings.warn(f"The column {col} is not numeric. Skipping.")
                continue
            stats = df_filtered[col].agg(['min', 'max', 'mean', 'std'])                 # sample statistics, most relevant if capping applied, else for reference only
            min_val, max_val = stats['min'], stats['max']                               # unadulterated min and max values for variable
            if cap:
                min_val, max_val = max(stats['mean'] - threshold * stats['std'], min_val), min(stats['mean'] + threshold * stats['std'], max_val) # capping applied if cap=True
            if debug:
                print(f"Scaling column {col}:")
                print(f"- Initial: Actual Min={stats['min']:.2f}, Max={stats['max']:.2f}. Capped Min={min_val:.2f}, Max={max_val:.2f}. Mean={stats['mean']:.2f}, Std={stats['std']:.2f}")
            scaled_series = pd.Series([0]*len(df)) if min_val == max_val else ((df[col] - min_val) / (max_val - min_val)).clip(0, 1)
            scaled_dict[col] = scaled_series                                            # series in the same order as the rows picked up from df, so can be manually added to the df later by assigning 
            if debug:
                print(f"- Scaled: Final Min={scaled_series.min():.2f}, Max={scaled_series.max():.2f}. Mean={scaled_series.mean():.2f}, Std={scaled_series.std():.2f}\n")
        return scaled_dict
    
    # initalizations
    sub_key = list(index_weight_dict[selected_index].keys())[0]         # intermediate sub key needed to pull metrics from perk_dict
    metrics = list(index_weight_dict[selected_index][sub_key].keys())   # list composition metrics to be scaled wrt index
    print(f"{selected_index}: {metrics}")
    scaled_dict = minmax(df=df, col_list=metrics, col_filter=col_filter, debug=debug, cap=cap, threshold=threshold)  # scale the metrics
    index_scores = df.copy()                                            # initialize dummy df for index scores
    for metric in metrics:                                              # weight scaled metrics
        scaled_series = scaled_dict[metric]  # individual series of scaled metrics
        weight = index_weight_dict[selected_index][sub_key][metric]  # draw relevant weight from perk_dict
        index_scores[f"{metric}_scaled_weighted"] = scaled_series * weight  # weight scaled metrics
    scaled_weighted_cols = [col for col in index_scores.columns if '_scaled_weighted' in col]  # name the columns
    index_scores[selected_index] = index_scores[scaled_weighted_cols].sum(axis=1)  # new column for index score as sum of weighted scaled metrics
    index_col = index_scores[selected_index]
    scaled_df = pd.DataFrame(scaled_dict)
    result_df = pd.DataFrame(pd.concat([scaled_df, index_col], axis=1))
    return result_df

# B. Data loading
* load full data set from source (full df) 

#### Express lane: import full df (if right version available) and skip rest of the section

In [ ]:
full = pd.read_csv(CONFIG_PATH["full"])                             # import full df if available
df_name_and_variable_sizes(full, ['user_id', 'session_id'])         # print counts of unique user_id and unique session_id
# df name is full ==> 1020926 user_id , 5408063 session_id

#### 1. Initial load data tables from postgres

In [ ]:
# prep
default_db_url = "postgres://Test:[redacted]/TravelTide"           # pre-filled database URL (redacted -  so it won't work if you run this)
db_url_input = widgets.Text(                                                                                        
    value=default_db_url,
    placeholder='Enter your DB URL',
    description='DB URL:',
    disabled=False)                                                                                                 # create a textbox for the database URL input
connect_button = widgets.Button(description="Connect and Execute Queries")      # create a button to execute the connection and queries
output = widgets.Output()                                                       # create an output widget to display messages

In [ ]:
def on_connect_button_clicked(b):                                               # run import process
    local_dataframes = {}                                                       # local variable to store DataFrames
    with output:
        clear_output(wait=True)                                                 # clear previous messages
        try:
            connection = psycopg2.connect(db_url_input.value)
            with connection.cursor() as cursor:
                cursor.execute("SELECT version();")
                version = cursor.fetchone()
                print("Connected to PostgreSQL version:", version)
                
                # get all table names in the public schema
                cursor.execute("""
                    SELECT table_name
                    FROM information_schema.tables
                    WHERE table_schema = 'public'
                """)
                table_names = [row[0] for row in cursor.fetchall()]
                
                for table_name in table_names:
                    query = f"SELECT * FROM {table_name};"
                    cursor.execute(query)
                    data = cursor.fetchall()
                    column_names = [desc[0] for desc in cursor.description]
                    df = pd.DataFrame(data, columns=column_names)
                    local_dataframes[table_name] = df
                
                connection.close()
                print("Queries executed successfully.")
                
        except psycopg2.Error as e:
            print("Error connecting to the database:", e)
    
    return local_dataframes                                                 # return populated dictionary
dataframes = on_connect_button_clicked(None)                                # pass None because this function expects a button argument, but we don't need it for manual execution

#### 2. Import tables as df and merge them to create full df

In [ ]:
print(dataframes.keys())                # names of loaded tables

In [ ]:
users = dataframes['users']             # based on printed keys, set up dfs of the tables
sessions = dataframes['sessions']
flights = dataframes['flights']
hotels = dataframes['hotels']

In [ ]:
full = pd.merge(users, sessions, on=['user_id'], how='left') # merge tables to create full df
full = pd.merge(full, flights, on=['trip_id'], how='left')
full = pd.merge(full, hotels, on=['trip_id'], how='left')

In [ ]:
full.to_csv(CONFIG_PATH["full"], index=False)               # export for future use

# C. Exogenous info

#### 1. Destination type info

In [ ]:
destinations = pd.DataFrame(full['destination'][~full['destination'].isna()].unique(), columns=['destination']) # initialize list of destinations from full df
df_name_and_variable_sizes(destinations, ['destination'])                                                       # print counts of destinations
# df name is destinations ==> 140 destination

In [ ]:
destination_mapping_dict = {                                                                                    # mapping dict, keys = city/destination
    # USA
    'los angeles': 'US', 'denver': 'US', 'dallas': 'US', 'houston': 'US', 'tucson': 'US', 'boston': 'US', 'washington': 'US', 'new york': 'US', 'chicago': 'US', 
    'jacksonville': 'US', 'detroit': 'US', 'indianapolis': 'US', 'philadelphia': 'US', 'charlotte': 'US', 'san antonio': 'US', 'miami': 'US', 'columbus': 'US', 'san jose': 'US', 
    'fort worth': 'US', 'seattle': 'US', 'phoenix': 'US', 'austin': 'US', 'portland': 'US', 'san diego': 'US', 'el paso': 'US', 'oklahoma city': 'US', 'fresno': 'US', 'louisville': 'US', 
    'milwaukee': 'US', 'nashville': 'US', 'las vegas': 'US', 'san francisco': 'US', 'baltimore': 'US', 'memphis': 'US', 'atlanta': 'US', 'orlando': 'US', 'honolulu': 'US',
    # Canada
    'ottawa': 'Canada', 'toronto': 'Canada', 'montreal': 'Canada', 'calgary': 'Canada', 'edmonton': 'Canada', 'winnipeg': 'Canada', 'quebec': 'Canada', 'hamilton': 'Canada', 'vancouver': 'Canada',
    # South America
    'rio de janeiro': 'Brazil', 'buenos aires': 'Argentina', 'lima': 'Peru', 'bogota': 'Colombia', 'quito': 'Ecuador', 'montevideo': 'Uruguay',
    # Mexico & Central America
    'mexico city': 'Mexico', 'havana': 'Cuba', 'punta cana': 'Dominican Republic',
    # Europe
    'stockholm': 'Sweden', 'copenhagen': 'Denmark', 'madrid': 'Spain', 'amsterdam': 'Netherlands', 'vienna': 'Austria', 'berlin': 'Germany', 'hamburg': 'Germany', 'london': 'UK', 
    'rome': 'Italy', 'prague': 'Czech Republic', 'paris': 'France', 'brussels': 'Belgium', 'barcelona': 'Spain', 'dublin': 'Ireland', 'warsaw': 'Poland', 'budapest': 'Hungary', 
    'moscow': 'Russia', 'lisbon': 'Portugal', 'milan': 'Italy', 'porto': 'Portugal', 'munich': 'Germany', 'geneva': 'Switzerland', 'florence': 'Italy', 'naples': 'Italy', 
    'edinburgh': 'UK', 'sofia': 'Bulgaria', 'bucharest': 'Romania', 'jerusalem': 'Israel', 'istanbul': 'Turkey', 'antalya': 'Turkey', 'heraklion': 'Greece', 'casablanca': 'Morocco', 
    'nice': 'France', 'venice': 'Italy',
    # Middle East
    'amman': 'Jordan', 'dubai': 'UAE', 'riyadh': 'Saudi Arabia', 'beirut': 'Lebanon', 'dammam': 'Saudi Arabia', 'abu dhabi': 'UAE',
    # Africa
    'cairo': 'Egypt', 'johannesburg': 'South Africa', 'durban': 'South Africa', 'cape town': 'South Africa', 'lagos': 'Nigeria', 'accra': 'Ghana', 'hurghada': 'Egypt',
    # Asia & OCeania
    'shanghai': 'China', 'bangalore': 'India', 'delhi': 'India', 'jaipur': 'India', 'singapore': 'Singapore', 'hong kong': 'China', 'macau': 'China', 'tokyo': 'Japan', 
    'beijing': 'China', 'jakarta': 'Indonesia', 'bangkok': 'Thailand', 'osaka': 'Japan', 'taipei': 'Taiwan', 'manila': 'Philippines', 'seoul': 'South Korea', 'kuala lumpur': 'Malaysia', 
    'hanoi': 'Vietnam', 'phuket': 'Thailand', 'ho chi minh city': 'Vietnam', 'denpasar': 'Indonesia', 'batam': 'Indonesia', 'tianjin': 'China', 'hangzhou': 'China', 'xiamen': 'China', 
    'chengdu': 'China', 'shenzhen': 'China', 'agra': 'India', 'guangzhou': 'China', 'johor bahru': 'Malaysia', 'fukuoka': 'Japan', 'dalian': 'China', 
    'guilin': 'China', "xi'an": 'China', 'pune': 'India', 'colombo': 'Sri Lanka', 'qingdao': 'China',
    'melbourne': 'Australia', 'sydney': 'Australia', 'auckland': 'New Zealand'}
find_unmapped_values(full, destination_mapping_dict, 'destination')                                             # check deficiency in destination_mapping_dict

In [ ]:
map_df= pd.DataFrame(list(destination_mapping_dict.items()), columns=['destination', 'destination_country'])                                            # df of mapping dict and merge into destinations df 
destinations = pd.merge(destinations, map_df, on=['destination'], how='left')                                                                           # merge mapping df into destinations df
destinations['destination_type'] = destinations['destination_country'].apply(lambda x: 'US & Canada' if x in ['US', 'Canada'] else 'foreign travel')    # new col destination_type, US & Canada vs foreign travel
destinations['foreign_destination'] = destinations['destination_type'] == 'foreign travel'                                                              # new col foreign_destination True/False/NaN

#### 2. Airline type info

In [ ]:
airlines = pd.DataFrame(full['trip_airline'].dropna().unique(), columns=['trip_airline'])   # initialize list of airlines from full df
df_name_and_variable_sizes(airlines, ['trip_airline'])                                      # print counts of airlines
# df name is airlines ==> 355 trip_airline

In [ ]:
budget_airlines_list = [                                                                    # list of known budget airlines up to 2021
    "Ryanair", "Wizz Air", "Frontier Airlines", "Spirit Airlines",
    "AirAsia", "AirAsia X", "IndiGo Airlines", "Jeju Air",
    "Jetstar Airways", "Jetstar Asia Airways","EasyJet",
    "Cebu Pacific", "Pegasus Airlines", "Tiger Airways", "Tiger Airways Australia",
    "Volaris", "Southwest Airlines", "Allegiant Air", "WestJet", "Norwegian Air Shuttle",
    "AirTran Airways", "Virgin America", "Flybe", "Nok Air", "Spicejet", "Go Air",
    "Lion Mentari Airlines", "Onur Air", "SunExpress", "Jet2.com",
    "Mango", "Rwandair Express", "Air Arabia", "Eurowings",
    "Peach", "Scoot", "WOW air", "Flydubai"]                                               # Peach, Scoot, WOW air and Flydubai absent from full df 

In [ ]:
airlines['budget_airline'] = airlines['trip_airline'].apply(lambda x: x in budget_airlines_list)        # budget airline True/False/NaN

#### 3. Hotel type info

In [ ]:
hotels = pd.DataFrame(full['hotel_name'][~full['hotel_name'].isna()].unique(), columns=['hotel_name'])                      # initialize list of hotel from full df
hotels[['hotel_brand', 'hotel_city']] = hotels['hotel_name'].str.split(' - ', expand=True).apply(lambda x: x.str.strip())   # expand=True to split 'hotel_name' into new cols and remove leading/trailing whitespaces with str.strip() in .apply(lambda x: x.) 
df_name_and_variable_sizes(hotels, ['hotel_name'])                                                                          # print counts of hotels
# df name is hotels ==> 2798 hotel_name

In [ ]:
ratings_dict = {                                                                                                            # hotel brand star ratings
    'Accor': {'rating': 'Varies', 'notes': 'Ranges from budget (e.g., Ibis) to luxury (e.g., Sofitel)', 'luxury_indicator': False},
    'Aman Resorts': {'rating': '5 star', 'notes': 'Luxury hotel chain', 'luxury_indicator': True},
    'Banyan Tree': {'rating': '5 star', 'notes': 'Luxury resort brand', 'luxury_indicator': True},
    'Best Western': {'rating': '2.5-4 star', 'notes': 'Varies with sub-brand like Best Western Plus', 'luxury_indicator': False},
    'Choice Hotels': {'rating': 'Varies', 'notes': 'Ranges from budget to upscale', 'luxury_indicator': False},
    'Conrad': {'rating': '5 star', 'notes': 'Hilton luxury brand', 'luxury_indicator': True},
    'Crowne Plaza': {'rating': '4 star', 'notes': 'Part of InterContinental Hotels Group', 'luxury_indicator': True},
    'Extended Stay': {'rating': '2.5-3.5 star', 'notes': 'Focuses on long-term stays', 'luxury_indicator': False},
    'Fairmont': {'rating': '5 star', 'notes': 'Luxury brand', 'luxury_indicator': True},
    'Four Seasons': {'rating': '5 star', 'notes': 'Renowned luxury brand', 'luxury_indicator': True},
    'Hilton': {'rating': '4 star', 'notes': 'Main brand; other brands vary from 3-5 stars', 'luxury_indicator': True},
    'Hyatt': {'rating': '4 star', 'notes': 'Main brand; other brands range from 3-5 stars', 'luxury_indicator': True},
    'InterContinental': {'rating': '5 star', 'notes': 'Luxury brand', 'luxury_indicator': True},
    'Marriott': {'rating': '4 star', 'notes': 'Main brand; has range from budget to luxury in other brands', 'luxury_indicator': True},
    'NH Hotel': {'rating': '3-4 star', 'notes': 'General rating', 'luxury_indicator': False},
    'Radisson': {'rating': '4 star', 'notes': 'Typical for main brand', 'luxury_indicator': True},
    'Rosewood': {'rating': '5 star', 'notes': 'Luxury brand', 'luxury_indicator': True},
    'Shangri-La': {'rating': '5 star', 'notes': 'Luxury brand', 'luxury_indicator': True},
    'Starwood': {'rating': 'Varies', 'notes': 'Had brands ranging from 3-5 stars. Acquired by Marriott in 2016.', 'luxury_indicator': False},
    'Wyndham': {'rating': '3-4 star', 'notes': 'Main brand rating; has a range from budget to upscale in other brands', 'luxury_indicator': False}}
find_unmapped_values(hotels, ratings_dict, 'hotel_brand')                                                                   # check deficiency in ratings_dict

In [ ]:
ratings_df = pd.DataFrame.from_dict(ratings_dict, orient='index').reset_index()         # df of ratings_dict, orient='index' to convert dict keys to index, reset_index so index keys is new colu 
ratings_df.columns = ['hotel_brand', 'hotel_star', 'notes', 'luxury_hotel']             # assign column names, luxury_hotel is bool
ratings_df = ratings_df.drop(columns=['notes'])                                         # drop notes column
df_name_and_variable_sizes(hotels, ['hotel_name', 'hotel_brand'])                       # print counts of hotel names and hotel brands in ratings_dict
# df name is hotels ==> 2798 hotel_name , 20 hotel_brand

In [ ]:
hotels = hotels.merge(ratings_df, on='hotel_brand', how='left')                         # merge ratings_df to hotels df
find_unmapped_values(hotels, destination_mapping_dict, 'hotel_city')                    # check deficiency in destination_mapping_dict to map hotel_city to country

In [ ]:
map_hotel_df= pd.DataFrame(list(destination_mapping_dict.items()), columns=['hotel_city', 'hotel_country'])                         # df of destination_mapping_dict but mapped to hotel_city
hotels = pd.merge(hotels, map_hotel_df, on=['hotel_city'], how='left')                                                              # merge mapping df into hotels df
hotels['hotel_location'] = hotels['hotel_country'].apply(lambda x: 'US & Canada' if x in ['US', 'Canada'] else 'foreign stay')      # new col if hotel is in US & Canada or a foreign stay
hotels['foreign_stay'] = hotels['hotel_location'] == 'foreign stay'                                                                 # new col foreign_stay True/False/NaN

# D. Cohort data
* Cohort full df, merge exogenous info and generate transaction related metrics (data df)

#### 0. Express lane: Import data df (if right version available) and skip rest of the section

In [19]:
# do not delete without care as will affect other sections
data_dtype_dict = {
    'datetime': ['birthdate', 'sign_up_date', 'session_start', 'session_end', 
                 'departure_time', 'return_time', 'check_in_time', 'check_out_time', 
                 'travel_start', 'travel_end', 'trip_booking_time'],
    'float': ['base_fare_usd_per_km', 'hotel_per_room_usd','search_session_clicks', 'search_session_mins'],
    'object': ['user_id']}

In [ ]:
data = pd.read_csv(CONFIG_PATH["data"])                                         # import data df if available
df_name_and_variable_sizes(data, ['user_id', 'session_id'])                     # print counts of unique user_id and unique session_id and get info
# df name is data ==> 5998 user_id , 49211 session_id

In [ ]:
convert_dtype_multi(data, dtype_dict=data_dtype_dict)

#### 1. Cohort data

In [ ]:
data = full.copy()                                                  # initialize data df by creating a copy on full df, make sure full df is loaded, and if not go to previous steps above to load
df_name_and_variable_sizes(data, ['user_id', 'session_id'])         # print counts of unique user_id and unique session_id and get info
# df name is data ==> 1020926 user_id , 5408063 session_id

In [ ]:
# reduce scope per Elena Tarrant request
data = data[data['session_start'] >= pd.Timestamp('2023-01-04 00:00:00')]        # filter for cut-off datetime 2023-01-04 00:00:00
data = data[data.groupby('user_id')['session_id'].transform('nunique') > 7]      # filter for users with more than 7 sessions, .transform() generates a Series (same length as df) each element is count of unique session_ids at groupby user_id level

In [ ]:
df_name_and_variable_sizes(data, ['user_id', 'session_id'])                             # print counts of unique user_id and unique session_id, should be lower than previous print
# df name is data ==> 5998 user_id , 49211 session_id

In [ ]:
# adhoc sense check

full.hotel_booked.dropna().count() # 5408063
full.trip_id.dropna().count() # 2335845
full.hotel_name.dropna().count() # 1962920
full[full.hotel_booked].hotel_name.dropna().count() # 1962920
full[full.trip_id.notna()].hotel_name.dropna().count() # 1962920
full[full.hotel_booked].hotel_booked.count() # 2009287, more rows with hotel_booked==True vs rows with non-NaN in hotel_name

data.trip_id.dropna().count() # 16702
data.hotel_booked.dropna().count() # 16702, includes False
data[data.hotel_booked==True].hotel_booked.count() # 14919 # number of rows for which hotel_booked is True
data.hotel_name.dropna().count() # 14726, again indicating some hotel_name rows are underpopulated

In [ ]:
data.to_csv(r"C:\\data\\tt\\py\\data_1.csv", index=False)                             # intermediate export as backup

#### 2. Merge exogenous info

In [ ]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_1.csv")

In [ ]:
data = pd.merge(data, destinations, on=['destination'], how='left') # merge destinations df
data = pd.merge(data, airlines, on=['trip_airline'], how='left') # merge airlines df
data = pd.merge(data, hotels, on=['hotel_name'], how='left') # merge hotels df

In [ ]:
# these columns are only applicable for where trip_id != NaN, outside of that it is NaN, some specific to flight_booked, hotel_booked and hotel_name
col_tidy = [# boolean type
            'flight_booked', 'hotel_booked', 'cancellation', 'return_flight_booked', 
            # travel times
            'departure_time', 'return_time', 'check_in_time', 'check_out_time','rooms', 
            # trip economics 
            'flight_discount', 'hotel_discount', 'seats', 'checked_bags', 'nights', 'base_fare_usd', 'hotel_per_room_usd', 
            # airport
            'home_airport_lat', 'home_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 
            # names 
            'origin_airport', 'destination', 'destination_airport', 'trip_airline', 'hotel_name']                                                                   # NaN if trip_id==NaN
data.loc[data['trip_id'].isna(), col_tidy] = np.nan
data.loc[data['trip_id'].isna() | data['flight_booked'].isna(), ['foreign_destination', 'budget_airline']] = np.nan                                           # restrict True/False variables to correct rows
data.loc[data['trip_id'].isna() | data['hotel_name'].isna() | data['hotel_booked'].isna(), ['luxury_hotel', 'foreign_stay']] = np.nan

In [ ]:
data.to_csv(r"C:\\data\\tt\\py\\data_2.csv", index=False)                             # intermediate export as backup

#### 3. Work around duplicated trip_id

In [ ]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_2.csv")

In [ ]:
# prep some cols to be the right dtype for calc in this section
trip_id_dtype_dict = {
    'datetime': ['birthdate', 'sign_up_date', 'session_start', 'session_end', 'departure_time', 'return_time', 'check_in_time', 'check_out_time'],
    'object': ['user_id', 'trip_id']}
convert_dtype_multi(data, dtype_dict=trip_id_dtype_dict)

In [ ]:
print(f"Max counts of session_start of trip_id: {(data.groupby('trip_id')['session_start'].nunique()).max()}") # some trip_ids have two session_starts max, one for origination, another cancellation, only most recent row eligible for metrics
# originating leg of some cancelled trip_id are not in the session_start cut-off requested by Elena 

In [ ]:
# Sort the DataFrame
data.sort_values(['trip_id', 'session_start'], inplace=True)

# Initialize 'use_trip_id' as NaN
data['use_trip_id'] = np.nan

# Set 'use_trip_id' as True for the most recent 'session_start' for each 'trip_id'
data.loc[data.groupby('trip_id')['session_start'].idxmax(), 'use_trip_id'] = True

# Find rows with only one session and set their 'use_trip_id' as True
single_session_mask = ~data['trip_id'].duplicated(keep=False)
data.loc[single_session_mask, 'use_trip_id'] = True

# Set 'use_trip_id' to False for other rows where 'trip_id' is not NaN
data.loc[data['trip_id'].notna() & data['use_trip_id'].isna(), 'use_trip_id'] = False

# Finally, set 'use_trip_id' to NaN where 'trip_id' is NaN
data.loc[data['trip_id'].isna(), 'use_trip_id'] = np.nan

In [ ]:
data.to_csv(r"C:\\data\\tt\\py\\data_3.csv", index=False)                             # intermediate export as backup

#### 4. Distances

In [176]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_3.csv")

C:\Users\Master\AppData\Local\Temp\ipykernel_51964\1526391504.py:1: DtypeWarning: Columns (12,15,16,19,20,22,23,24,25,27,28,29,31,35,38,39,41,42,43,44,45,46,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\data\\tt\\py\\data_3.csv")


In [177]:
# prep some cols to be the right dtype for calc in this section
distances_dtype_dict = {
    'float': ['home_airport_lat', 'home_airport_lon', 'destination_airport_lat', 'destination_airport_lon'],
    'object': ['user_id']}
convert_dtype_multi(data, dtype_dict=distances_dtype_dict)

In [178]:
# generate distance cols, includes cancelled trips, use karney method for calcs for metrics
distances(data, lat1_col='home_airport_lat', lon1_col='home_airport_lon', lat2_col='destination_airport_lat', lon2_col='destination_airport_lon',
              flight_booked_col='flight_booked', return_flight_col='return_flight_booked', method='karney', 
              short_haul_max=1287, medium_haul_max=4023, long_haul_max=10460)

C:\Users\Master\AppData\Roaming\Python\Python310\site-packages\IPython\lib\pretty.py:778: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  output = repr(obj)
C:\Users\Master\AppData\Roaming\Python\Python310\site-packages\IPython\core\formatters.py:344: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  return method()


,user_id,birthdate,gender,married,has_children,home_country,home_city,home_airport,home_airport_lat,home_airport_lon,...,foreign_stay,use_trip_id,flown_km_haversine_basic,flown_km_haversine_oblate,flown_km_karney,distance_method,flown_km,distance_km,flight_category,flight_long_haul
0,101486,1972-12-07,F,True,True,usa,tacoma,TCM,47.138,-122.476,...,False,True,965.340568,963.184958,966.850557,karney,1933.701114,966.850557,short haul,False
1,101486,1972-12-07,F,True,True,usa,tacoma,TCM,47.138,-122.476,...,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005,...,False,True,300.162123,299.491599,300.479240,karney,600.958480,300.479240,short haul,False
3,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005,...,False,True,400.220489,399.326543,400.244156,karney,800.488312,400.244156,short haul,False
4,101961,1980-09-14,F,True,False,usa,boston,BOS,42.364,-71.005,...,False,True,4003.264513,3994.323080,4014.164906,karney,8028.329812,4014.164906,medium haul,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49206,533809,1985-11-06,F,True,True,usa,milwaukee,MKE,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,karney,NaN,NaN,None,None
49207,621849,1971-06-15,F,True,True,usa,birmingham,BHM,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,karney,NaN,NaN,None,None
49208,599790,1967-07-23,F,False,False,usa,new york,JFK,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,karney,NaN,NaN,None,None
49209,435596,1970-11-28,F,False,False,usa,houston,EFD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,karney,NaN,NaN,None,None


In [179]:
# distance_method should only have values where distance_km col has values
data.loc[data['distance_km'].isna() , ['distance_method']] = np.nan                     # tidy up
data = data.drop(columns=['flown_km_haversine_basic', 'flown_km_haversine_oblate', 'flown_km_karney']) # drop useless cols

In [180]:
data.to_csv(r"C:\\data\\tt\\py\\data_4.csv", index=False)                               # intermediate export as backup

#### 5. Boolean indicators

In [181]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_4.csv")

C:\Users\Master\AppData\Local\Temp\ipykernel_51964\3486984536.py:1: DtypeWarning: Columns (12,15,16,19,20,22,23,24,25,27,28,29,31,35,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\data\\tt\\py\\data_4.csv")


In [28]:
# general and product mix indicators
booking = (data['trip_id'].notna()) & (data['cancellation'].notna()) & (data['use_trip_id']==True)
flight_booking = booking & (data['flight_booked']==True)
hotel_booking = booking & (data['hotel_booked']==True)
combo_booking = flight_booking & hotel_booking

cancellation = booking & (data['cancellation']==True) 
trip = booking & (data['cancellation']==False)

flight_trip = trip & flight_booking
hotel_trip = trip & hotel_booking 
combo_trip = trip & combo_booking
return_trip = flight_trip & (data['return_flight_booked']==True)

flight_only_trip = flight_trip & (data['hotel_booked']==False)
hotel_only_trip = hotel_trip & (data['flight_booked']==False)

# behavioral indicators
foreign_trip = trip & ((data['foreign_stay']==True) | (data['foreign_destination']==True))
long_haul_trip = flight_trip & (data['flight_long_haul']==True)
staycation = hotel_trip & (data['home_city'] == data['hotel_city'])

budget_flight = flight_trip & (data['budget_airline']==True)
premium_flight = flight_trip & (data['budget_airline']==False)
luxury_stay = hotel_trip & (data['luxury_hotel']==True)
quality_travel = luxury_stay | premium_flight

# discount indicators
discount_trip = trip & ((data['flight_discount']==True) | (data['hotel_discount']==True))
discount_flight_trip = flight_trip & (data['flight_discount']==True)  
discount_hotel_trip = hotel_trip & (data['hotel_discount']==True)

In [29]:
# non cancellation session
non_cancellation_session = ~(data['cancellation']==True)

In [184]:
# eligible denonminators
data['booking'] = booking                          
data['cancelled_booking'] = cancellation
data['trip'] = trip
# general product behavior     
data['trip_with_flight'] = flight_trip
data['trip_with_hotel'] = hotel_trip
data['combo_trip'] = combo_trip                             
# exclusive product behavior
data['flight_only_trip'] = flight_only_trip
data['hotel_only_trip'] = hotel_only_trip
# other behavior
data['return_trip'] = return_trip
data['foreign_trip'] = foreign_trip
data['long_haul_trip'] = long_haul_trip
data['staycation'] = staycation
# propensity to spend
data['budget_flight'] = budget_flight
data['premium_flight'] = premium_flight
data['luxury_stay'] = luxury_stay
data['quality_travel'] = quality_travel
# bargain hunting
data['discount_trip'] = discount_trip
data['discount_flight_trip'] = discount_flight_trip
data['discount_hotel_trip'] = discount_hotel_trip

In [185]:
# these booleans are only applicable for where trip_id != NaN, outside of that it is NaN
data.loc[data['trip_id'].isna() , ['booking', 'cancelled_booking', 'trip',                              # eligible denonminators
                                   'trip_with_flight', 'trip_with_hotel', 'combo_trip',                 # general product behavior 
                                   'flight_only_trip', 'hotel_only_trip',                               # exclusive product behavior
                                   'return_trip', 'foreign_trip', 'long_haul_trip', 'staycation',       # other behavior
                                    'budget_flight', 'premium_flight', 'luxury_stay', 'quality_travel', # propensity to spend
                                    'discount_trip', 'discount_flight_trip', 'discount_hotel_trip'      # bargain hunting
                                    ]] = np.nan                                                         # tidy up

In [186]:
data.to_csv(r"C:\\data\\tt\\py\\data_5.csv", index=False)                             # intermediate export as backup

#### 6. Periods and durations

In [64]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_5.csv")

C:\Users\Master\AppData\Local\Temp\ipykernel_58676\1410142805.py:1: DtypeWarning: Columns (12,15,16,19,20,22,23,24,25,27,28,29,31,35,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\data\\tt\\py\\data_5.csv")


In [65]:
# prep some cols to be the right dtype for calc in this section
periods_dtype_dict = {
    'datetime': ['birthdate', 'sign_up_date', 'session_start', 'session_end', 'departure_time', 'return_time', 'check_in_time', 'check_out_time'],
    'float': ['hotel_per_room_usd'],
    'bool': ['married'],
    'object': ['user_id']}
convert_dtype_multi(data, dtype_dict=periods_dtype_dict)

In [66]:
data['session_duration_mins'] = (data['session_end'] - data['session_start']).dt.total_seconds() / 60  # copnvert to mins float

In [67]:
data['search_session_mins'] = 0
data.loc[non_cancellation_session, 'search_session_mins'] = data['session_duration_mins']

In [68]:
data['search_session_clicks'] = 0
data.loc[non_cancellation_session, 'search_session_clicks'] = data['page_clicks']

In [69]:
# monetizing bookings only
data.loc[trip, 'travel_start'] = data[['departure_time', 'check_in_time']].min(axis=1).where((data['departure_time'].notna() & data['check_in_time'].notna()), other=data['departure_time'].combine_first(data['check_in_time']))
data.loc[trip, 'travel_end'] = data[['return_time', 'check_out_time']].max(axis=1).where((data['return_time'].notna() & data['check_out_time'].notna()), other=data['return_time'].combine_first(data['check_out_time']))
data.loc[trip, 'travel_period'] = data['travel_start'].dt.to_period('M')
data.loc[trip, 'travel_days'] = np.ceil((data['travel_end'] - data['travel_start']).dt.total_seconds() / (24 * 60 * 60))                           # convert days float, duration of travel, small num of trips 1-way flights as travel_end is NaN

data.loc[trip, 'trip_booking_time'] = pd.to_datetime(data['session_end'])                                                                           # time of booking non-cancelled trips
data.loc[trip, 'days_pre_travel'] = np.ceil((data.loc[trip, 'travel_start'] - data.loc[trip, 'trip_booking_time']).dt.total_seconds() / (24 * 60 * 60))  # division to convert to days float, non-cancelled

data.loc[flight_trip, 'flight_period'] = data['departure_time'].dt.to_period('M')                      # booked flight month-year periods
data.loc[return_trip, 'days_between_flights'] = np.ceil((data['return_time'] - data['departure_time']).dt.total_seconds() / (24 * 60 * 60))    # convert days float, return_time NaN if no return flight booked

data.loc[hotel_trip,'hotel_period'] = data['check_in_time'].dt.to_period('M')                          # booked hotel month-year periods

data.loc[hotel_trip,'long_stay'] = data['nights'] > 7
data.loc[trip, 'long_travel'] = data['travel_days'] > 7

data.loc[trip, 'early_booking'] = data['days_pre_travel'] >= 180
data.loc[trip, 'late_booking'] = data['days_pre_travel'] <= 7
data.loc[trip, 'booking_60_to_180_days'] = (data['days_pre_travel'] >= 60) & (data['days_pre_travel'] < 180) 

In [70]:
# these booleans are only applicable for where trip_id != NaN, outside of that it is NaN
data.loc[data['trip_id'].isna() , ['travel_start', 'travel_end', 'travel_period', 'travel_days',
                                       'trip_booking_time', 'days_pre_travel',
                                       'flight_period', 'days_between_flights',
                                       'hotel_period', 'long_stay', 'long_travel', 
                                       'early_booking', 'late_booking', 'booking_60_to_180_days'
                                    ]] = np.nan                                                         # tidy up

In [71]:
data.to_csv(r"C:\\data\\tt\\py\\data_6.csv", index=False)                             # intermediate export as backup

#### 7. Economics

In [72]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_6.csv")

C:\Users\Master\AppData\Local\Temp\ipykernel_58676\1542130171.py:1: DtypeWarning: Columns (12,15,16,19,20,22,23,24,25,27,28,29,31,35,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,84,86,88,89,90,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\data\\tt\\py\\data_6.csv")


In [73]:
# prep some cols to be the right dtype for calc in this section
economics_dtype_dict = {
    'datetime': ['birthdate', 'sign_up_date', 'session_start', 'session_end', 
                 'departure_time', 'return_time', 'check_in_time', 'check_out_time', 
                 'travel_start', 'travel_end', 'trip_booking_time'],
    'float': ['flight_discount_amount', 'hotel_discount_amount', 'hotel_per_room_usd', 'distance_km', 'base_fare_usd', 'seats', 'rooms', 'nights', 'checked_bags'],
    'bool': ['married'],
    'object': ['user_id']}
convert_dtype_multi(data, dtype_dict=economics_dtype_dict)

In [74]:
# these booleans are only applicable for where trip_id != NaN, outside of that it is NaN
data.loc[data['trip_id'].isna() , ['booking', 'cancelled_booking', 'trip',                              # eligible denonminators
                                   'trip_with_flight', 'trip_with_hotel', 'combo_trip',                 # general product behavior 
                                   'flight_only_trip', 'hotel_only_trip',                               # exclusive product behavior
                                   'return_trip', 'foreign_trip', 'long_haul_trip', 'staycation',       # other behavior
                                    'budget_flight', 'premium_flight', 'luxury_stay', 'quality_travel', # propensity to spend
                                    'discount_trip', 'discount_flight_trip', 'discount_hotel_trip',     # bargain hunting
                                    'early_booking', 'late_booking', 'booking_60_to_180_days'           # pre-booking     
                                    ]] = np.nan                                                         # tidy up

In [75]:
# estimate travellers
data['num_travellers'] = data.apply(
    lambda row: 
        max(row['rooms'], row['seats']) if (row['trip'] == True) and pd.notna(row['rooms']) and pd.notna(row['seats'])         # higher of two, which likely to be seat
        else row['rooms'] if (row['trip'] == True) and pd.notna(row['rooms'])                                                  # more accurate since one person per seat
        else row['seats'] if (row['trip'] == True) and pd.notna(row['seats'])                                                 # relying on this alone may underestimate travellers due to couples or groups traveling
        else np.nan, 
    axis=1
)

In [76]:
# rates and intensities
data['bags_per_seat'] = np.where(                                                    
    (data['seats'] != 0) & (data['seats'].notna()),
    data['checked_bags'] / data['seats'],
    np.nan)

data['guests_per_room'] = np.where(                                                    
    (data['rooms'] != 0) & (data['rooms'].notna()),
    data['num_travellers'] / data['rooms'],
    np.nan)

data['base_fare_usd_per_km'] = np.where(                                                    
    (data['distance_km'] != 0) & (data['distance_km'].notna()),
    data['base_fare_usd'] / data['distance_km'],
    np.nan)

In [77]:
# gross value of flight booking - net of cancellation
data['flight_gross_value_usd'] = np.where(                                                    
    flight_trip & data['base_fare_usd'].notna() & data['seats'].notna(), 
    data['base_fare_usd'] * data['seats'],
    0)

# flight discount amount in usd
data['flight_discount_usd'] = np.where(                                                    
    discount_flight_trip & data['flight_discount_amount'].notna(),
    data['flight_gross_value_usd'] * data['flight_discount_amount'], 
    0)

data['flight_actual_cost_usd'] = data['flight_gross_value_usd'] - data['flight_discount_usd']   # actual flight cost in usd

In [78]:
# room nights booked - net of cancellation
data['room_nights'] = np.where(
    hotel_trip &  (data['rooms']>=0) & (data['nights']>=0),
    data['rooms'] * data['nights'],
    0)

# gross value of hotel booking - net of cancellation
data['hotel_gross_value_usd'] = np.where(                                                    
    hotel_trip & data['hotel_per_room_usd'].notna(), 
    data['room_nights'] * data['hotel_per_room_usd'],
    0)

# hotel discount amount in usd
data['hotel_discount_usd'] = np.where(                                                    
    discount_hotel_trip & data['hotel_discount_amount'].notna(),
    data['hotel_gross_value_usd'] * data['hotel_discount_amount'], 
    0)

data['hotel_actual_cost_usd'] = data['hotel_gross_value_usd'] - data['hotel_discount_usd']  # actual hotel cost in usd

In [79]:
data['trip_gross_value_usd'] = data['flight_gross_value_usd'] + data['hotel_gross_value_usd']       # gross value of trip - net of cancellation
data['trip_discount_usd'] = data['flight_discount_usd'] + data['hotel_discount_usd']                # trip discount amount in usd
data['trip_actual_cost_usd'] = data['flight_actual_cost_usd'] + data['hotel_actual_cost_usd']       # actual trip cost in usd

In [80]:
data['value_per_traveller'] = data['trip_actual_cost_usd'] / data['num_travellers']

In [81]:
data['value_per_seat_per_km'] = (data['flight_actual_cost_usd'] / data['seats'] ) / data['distance_km']

In [82]:
data['travel_location'] = np.where(data['hotel_city'].notna(), data['hotel_city'], 
    np.where(data['destination'].notna(), data['destination'], 
        np.nan))                                                                                    # equivalent to multi-blanket if statement 

In [83]:
# time i.e. travel period and locations matter in determining costs of flight, hotel
# below looks at relative discount ratio of cost of flight, hotel or overall trip to that average for same time and location
# at overall trip level, adjustment is not made for single product bookings, though it is assumed single product bookings reflect greater frugality
# goal is to estimate frugality / bargain hunting behavior, controlled for time and location

# for flight
data.loc[flight_trip, 'comparable_flight_discount_ratio'] = 1 - (data['flight_actual_cost_usd'].div(
    data.groupby(['flight_period', 'destination'])['flight_actual_cost_usd']
    .transform('mean')))

# equivalent below for hotel cost
data.loc[hotel_trip, 'comparable_hotel_discount_ratio'] = 1 - (data['hotel_actual_cost_usd'].div(
    data.groupby(['hotel_period', 'hotel_city'])['hotel_actual_cost_usd']
    .transform('mean')))

# equivalent below for overall trip cost
data.loc[trip, 'comparable_trip_discount_ratio'] = 1 - (data['trip_actual_cost_usd'].div(
    data.groupby(['travel_period', 'travel_location'])['trip_actual_cost_usd']
    .transform('mean')))

In [84]:
data['days_cut_off_vs_sign_up'] = np.ceil((data['session_end'].max() - data['sign_up_date']).dt.total_seconds() / (24 * 60 * 60)) # convert days float

In [85]:
data.to_csv(r"C:\\data\\tt\\py\\data_7.csv", index=False)                             # intermediate export as backup

#### 8. Finalize

In [149]:
data = pd.read_csv(r"C:\\data\\tt\\py\\data_7.csv")

C:\Users\Master\AppData\Local\Temp\ipykernel_58676\2715002540.py:1: DtypeWarning: Columns (12,15,16,19,20,22,23,24,25,27,28,29,31,35,38,39,41,42,43,44,45,46,47,48,49,50,51,52,53,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,80,81,82,84,86,88,89,90,91,92,93,110) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(r"C:\\data\\tt\\py\\data_7.csv")


In [150]:
pd.DataFrame(list(data.columns), columns = ['var']).to_csv(r"C:\\data\\tt\\py\\columns.csv", index=False)  # check cols manually 

In [151]:
data_var = [
    'user_id', 'birthdate', 'home_city', 'sign_up_date', 'days_cut_off_vs_sign_up', # user personal details
    
    'session_id', 'page_clicks', 'search_session_clicks', 'session_start', 'session_end', 'session_duration_mins', 'search_session_mins', # session metrics

    'trip_id', 'use_trip_id', 'booking', 'cancelled_booking',                                                       # general bookings and cancellations

    'trip', 'combo_trip', 'discount_trip', 'foreign_trip', 'quality_travel', 'travel_location',                     # monetizing bookings - general
    'travel_start', 'travel_end', 'travel_period', 'travel_days', 'long_travel',                                    # monetizing bookings - travel duration
    'trip_booking_time', 'days_pre_travel', 'early_booking', 'late_booking', 'booking_60_to_180_days',              # monetizing bookings - prebooking tendency
    
    'flight_period', 'trip_with_flight', 'flight_only_trip', 'discount_flight_trip', 'flight_category', 'long_haul_trip', 'return_trip', 'budget_flight', 'premium_flight', # flights
    'trip_airline', 'flight_discount_amount', 'departure_time', 'return_time',                                                      # flights
    'home_airport_lat', 'home_airport_lon', 'destination_airport_lat', 'destination_airport_lon', 'destination',                    # flights

    'hotel_period', 'trip_with_hotel', 'hotel_only_trip', 'discount_hotel_trip', 'long_stay', 'staycation', 'luxury_stay',      # hotels
    'hotel_brand', 'hotel_discount_amount',  'check_in_time', 'check_out_time', 'hotel_city',                                   # hotels
    
    'flight_gross_value_usd', 'flight_discount_usd', 'flight_actual_cost_usd',                                      # economics
    'hotel_gross_value_usd', 'hotel_discount_usd', 'hotel_actual_cost_usd',                                         # economics
    'trip_gross_value_usd', 'trip_discount_usd', 'trip_actual_cost_usd',                                            # economics
    
    'num_travellers',                                                                                               # product and economic drivers
    'guests_per_room', 'nights', 'rooms', 'room_nights',                                                            # product and economic drivers
    'base_fare_usd', 'seats', 'bags_per_seat', 'checked_bags', 'distance_km',                                       # product and economic drivers
    
    'base_fare_usd_per_km', 'hotel_per_room_usd',                                                                   # economic intensities

    'value_per_traveller', 'value_per_seat_per_km',                                                                 # customer value intensities
    'comparable_flight_discount_ratio', 'comparable_hotel_discount_ratio', 'comparable_trip_discount_ratio'         # discount intensities
    ]

In [152]:
data = data[data_var]

In [153]:
convert_dtype_multi(data, dtype_dict=data_dtype_dict)

In [154]:
# Export for future ise
data.to_csv(CONFIG_PATH["data"], index=False)

# E. Clean and remove outliers
* Create a clean version of data df (clean df)

In [155]:
# prep
data = pd.read_csv(CONFIG_PATH["data"]) # import data df if available
convert_dtype_multi(data, dtype_dict=data_dtype_dict)
df_name_and_variable_sizes(data, ['user_id', 'session_id'])            # print counts of unique user_id and unique session_id
# df name is data ==> 5998 user_id , 49211 session_id

C:\Users\Master\AppData\Local\Temp\ipykernel_58676\2842690260.py:2: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,49,50,51,52,53,54,55,56,57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(CONFIG_PATH["data"]) # import data df if available


df name is data ==> 5998 user_id , 49211 session_id


In [156]:
precut = data.loc[~(data['nights'] < 0)].copy()    # .loc to make sure we work with rows where 'nights' are non-negative
df_name_and_variable_sizes(precut, ['user_id', 'session_id'])       # print counts of unique user_id and unique session_id
# df name is precut ==> 5998 user_id , 49104 session_id

df name is precut ==> 5998 user_id , 49104 session_id


In [157]:
economic_intensivity = ['hotel_per_room_usd', 'value_per_seat_per_km']            # intensitivy of trip economics to test outliers

In [158]:
data_2_stdev, data_ex_2_stdev = remove_outliers(precut, col_list=economic_intensivity, col_filter='use_trip_id', reference_col='session_id', 
                    methodology='stdev', threshold=2, whisker_size=0.5) # stdev method, no impact on calculation from IQR whisker 

df_name_and_variable_sizes(data_2_stdev, ['user_id', 'session_id'])    # print counts of unique user_id and unique session_id
# df name is data_2_stdev ==> 5998 user_id , 47980 session_id

Selected cols for stdev: ['hotel_per_room_usd', 'value_per_seat_per_km']
df name is data_2_stdev ==> 5998 user_id , 47980 session_id


In [159]:
data_IQR_whisker, data_ex_IQR_whisker = remove_outliers(precut, col_list=economic_intensivity, col_filter='use_trip_id', reference_col='session_id', 
                    methodology='iqr', threshold=2, whisker_size=0.5) # iqr method, no impact on calculation from stdev threshold

df_name_and_variable_sizes(data_IQR_whisker, ['user_id', 'session_id'])    # print counts of unique user_id and unique session_id
# df name is data_IQR_whisker ==> 5998 user_id , 42822 session_id

Selected cols for iqr: ['hotel_per_room_usd', 'value_per_seat_per_km']
df name is data_IQR_whisker ==> 5998 user_id , 42822 session_id


In [160]:
clean = data_2_stdev.copy() # assign to the choice of data sets above
outliers = data_ex_2_stdev.copy() # assign to the choice of data sets above

In [161]:
clean.to_csv(CONFIG_PATH["clean"], index=False)       # export for future use
outliers.to_csv(CONFIG_PATH["outliers"], index=False)       # export for future use

# F. User behavioral metrics    
* Create user metrics (user df)

#### Express lane: Import clean df (if right version available)

In [162]:
clean = pd.read_csv(CONFIG_PATH["clean"])                        # import data df if available
clean_dtype_dict = data_dtype_dict
convert_dtype_multi(clean, dtype_dict=clean_dtype_dict)          # clean and data df have same columns hence same columns need prepping for correct dtype
df_name_and_variable_sizes(clean, ['user_id', 'session_id'])     # print counts of unique user_id and unique session_id
# df name is clean ==> 5998 user_id , 47677 session_id

C:\Users\Master\AppData\Local\Temp\ipykernel_58676\3023440181.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,49,50,51,52,53,54,55,56,57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  clean = pd.read_csv(CONFIG_PATH["clean"])                        # import data df if available


df name is clean ==> 5998 user_id , 47980 session_id


In [148]:
essential_var = [ 
    'user_id', 'home_city', 'sign_up_date', 'days_cut_off_vs_sign_up', # user personal details
    
    'session_id', 'page_clicks', 'search_session_clicks', 'session_duration_mins', 'search_session_mins',   # session metrics

    'trip_id', 'use_trip_id', 'booking', 'cancelled_booking',                                               # general bookings and cancellations

    'travel_location', 'travel_period', 'travel_days',                                                      # monetizing bookings - general
    'trip', 'combo_trip', 'discount_trip', 'foreign_trip', 'quality_travel', 'long_travel',                 # monetizing bookings - general
    
    'trip_booking_time', 'days_pre_travel', 'early_booking', 'late_booking', 'booking_60_to_180_days',      # monetizing bookings - prebooking tendency
        
    'destination',  'flight_period',                                            # flights
    'trip_with_flight', 'flight_only_trip', 'discount_flight_trip',             # flights
    'long_haul_trip', 'return_trip', 'budget_flight', 'premium_flight',         # flights
    
    'hotel_city',   'hotel_period',                                             # hotels
    'trip_with_hotel', 'hotel_only_trip', 'discount_hotel_trip',                # hotels
    'long_stay', 'staycation', 'luxury_stay',                                   # hotels
    
    'flight_discount_amount', 'hotel_discount_amount',    
    'flight_gross_value_usd', 'flight_discount_usd', 'flight_actual_cost_usd',                                      # economics
    'hotel_gross_value_usd', 'hotel_discount_usd', 'hotel_actual_cost_usd',                                         # economics
    'trip_gross_value_usd', 'trip_discount_usd', 'trip_actual_cost_usd',                                            # economics

    'num_travellers',                                                                                               # product and economic drivers
    'guests_per_room', 'nights', 'rooms', 'room_nights',                                                            # product and economic drivers
    'base_fare_usd', 'seats', 'bags_per_seat', 'checked_bags', 'distance_km',                                       # product and economic drivers

    'base_fare_usd_per_km', 'hotel_per_room_usd',                                                                   # economic intensities

    'value_per_traveller', 'value_per_seat_per_km',                                                                 # customer value intensities
    'comparable_flight_discount_ratio', 'comparable_hotel_discount_ratio', 'comparable_trip_discount_ratio'         # discount intensities
]

In [165]:
# Check for repeats
if len(essential_var) != len(set(essential_var)):
    print("There are repeated strings.")

    # Identify repeated strings
    from collections import Counter
    counts = Counter(essential_var)
    repeated = [item for item, count in counts.items() if count > 1]
    print(f"Repeated strings are: {repeated}")
else:
    print("No repeated strings.")

No repeated strings.


#### 1. User metrics (user df) 

In [252]:
group = (clean[essential_var]).groupby('user_id')   # group df
user = pd.DataFrame()                               # initialize empty df for user metrics

In [253]:
# denominators - transaction counts
user['sessions'] = group['session_id'].nunique()
user['searches'] = group.apply(lambda x: x[(x['search_session_clicks'].notna()) & (x['search_session_mins'].notna())]['session_id'].nunique())
user['bookings'] = group.apply(lambda x: x[x['booking']==True]['trip_id'].nunique())
user['non_booking_sessions'] = user['searches'] - user['bookings']
user['cancellations'] = group.apply(lambda x: x[x['cancelled_booking']==True]['trip_id'].nunique())                   
user['trips'] = group.apply(lambda x: x[x['trip']==True]['trip_id'].nunique())
user['flight_trip'] = group.apply(lambda x: x[x['trip_with_flight']==True]['trip_id'].nunique())               
user['hotel_trip'] = group.apply(lambda x: x[x['trip_with_hotel']==True]['trip_id'].nunique())
user['flight_only_trip'] = group.apply(lambda x: x[x['flight_only_trip']==True]['trip_id'].nunique())          
user['hotel_only_trip'] = group.apply(lambda x: x[x['hotel_only_trip']==True]['trip_id'].nunique())
# product volumes
user['travellers'] = group.apply(lambda x: x[x['trip']==True]['num_travellers'].sum())
user['seats'] = group.apply(lambda x: x[x['trip_with_flight']==True]['seats'].sum())
user['checked_bags'] = group.apply(lambda x: x[x['trip_with_flight']==True]['checked_bags'].sum())
user['hotel_guests'] = group.apply(lambda x: x[x['trip_with_hotel']==True]['num_travellers'].sum())
user['nights'] = group.apply(lambda x: x[x['trip_with_hotel']==True]['nights'].sum())
user['rooms'] = group.apply(lambda x: x[x['trip_with_hotel']==True]['nights'].sum())
user['room_units'] = group.apply(lambda x: x[x['trip_with_hotel']==True]['room_nights'].sum())
# value - usd
user['gmv'] = group.apply(lambda x: x[x['trip']==True]['trip_gross_value_usd'].sum())           # total customer gross value pre  discount
user['savings'] = group.apply(lambda x: x[x['trip']==True]['trip_discount_usd'].sum())          # total customer savings from discount
user['tcv'] = group.apply(lambda x: x[x['trip']==True]['trip_actual_cost_usd'].sum())           # total customer value 
user['flight_tcv'] = group.apply(lambda x: x[x['trip']==True]['flight_actual_cost_usd'].sum())  # total customer value 
user['hotel_tcv'] = group.apply(lambda x: x[x['trip']==True]['hotel_actual_cost_usd'].sum())    # total customer value 
# seasoning
user['days_cut_off_vs_sign_up'] = group['days_cut_off_vs_sign_up'].mean()
user['avg_days_pre_travel'] = group['days_pre_travel'].mean()

In [254]:
# drop at the end
# general trips
user['combo_trip'] = group.apply(lambda x: x[x['combo_trip']==True]['trip_id'].nunique())                      
user['foreign_trip'] = group.apply(lambda x: x[x['foreign_trip']==True]['trip_id'].nunique())                                            
user['discount_trip'] = group.apply(lambda x: x[x['discount_trip']==True]['trip_id'].nunique())                
user['quality_travel'] = group.apply(lambda x: x[x['quality_travel']==True]['trip_id'].nunique())              

# drop at the end
# flight trips
user['long_haul_trip'] = group.apply(lambda x: x[x['long_haul_trip']==True]['trip_id'].nunique())              
user['budget_flight'] = group.apply(lambda x: x[x['budget_flight']==True]['trip_id'].nunique())                
user['premium_flight'] = group.apply(lambda x: x[x['premium_flight']==True]['trip_id'].nunique())              
user['extended_return_trip'] = group.apply(lambda x: x[(x['return_trip']==True) & (x['long_travel']==True)]['trip_id'].nunique())

# drop at the end
# hotel trips
user['staycation'] = group.apply(lambda x: x[x['staycation']==True]['trip_id'].nunique())                            
user['long_stay'] = group.apply(lambda x: x[x['long_stay']==True]['trip_id'].nunique())  

# drop at the end
# pre-booking tendency
user['early_booking'] = group.apply(lambda x: x[x['early_booking']==True]['trip_id'].nunique())  
user['late_booking'] = group.apply(lambda x: x[x['late_booking']==True]['trip_id'].nunique())
user['booking_60_to_180_days'] = group.apply(lambda x: x[x['booking_60_to_180_days']==True]['trip_id'].nunique())

In [255]:
user['monetization_rate'] = user['trips'].div(user['searches'])                                     # preferred perks (free hotel meal, free checked bag, one night free hotel with flight)
# user['trips']                                                                                     # preferred perks 
# user['days_cut_off_vs_sign_up']                                                                   # preferred perks
user['travellers_per_trip'] = group.apply(lambda x: x[x['trip']==True]['num_travellers'].mean())    # preferred perks
user['tcv_per_traveller'] = user['tcv'].div(user['travellers'])                                     # preferred perks
user['tcv_per_room_unit'] = user['tcv'].div(user['room_units'])                                     # preferred perks
user['avg_tcv_per_seat_per_km'] = group['value_per_seat_per_km'].mean()                             # preferred perks

In [256]:
user['quality_travel_ratio'] = user['quality_travel'].div(user['trips'])                # free hotel meal
user['hotel_guests_per_trip'] = user['hotel_guests'].div(user['hotel_trip'])            # free hotel meal
user['long_stay_ratio'] = user['long_stay'].div(user['hotel_trip'])                     # free hotel meal
user['staycation_ratio'] = user['staycation'].div(user['hotel_trip'])                   # free hotel meal
user['foreign_trip_ratio'] = user['foreign_trip'].div(user['trips'])                    # free hotel meal
user['long_haul_ratio'] = user['long_haul_trip'].div(user['flight_trip'])               # free hotel meal

In [257]:
user['premium_flight_ratio'] = user['premium_flight'].div(user['flight_trip'])               # one night free hotel with flight
user['flight_only_ratio'] = user['flight_only_trip'].div(user['trips'])                      # one night free hotel with flight
user['seats_per_trip'] = user['seats'].div(user['flight_trip'])                              # one night free hotel with flight
user['extended_return_trip_ratio'] = user['extended_return_trip'].div(user['flight_trip'])   # one night free hotel with flight
# user['foreign_trip_ratio']                                                                 # one night free hotel with flight
# user['long_haul_ratio']                                                                    # one night free hotel with flight

In [258]:
# user['seats_per_trip']                                                                # free checked bag
user['max_bags_per_seat'] = group.apply(lambda x: x[x['trip_with_flight']==True]['bags_per_seat'].max()) # free checked bag   
user['budget_flight_ratio'] = user['budget_flight'].div(user['flight_trip'])            # free checked bag
# user['extended_return_trip_ratio']                                                    # free checked bag
# user['foreign_trip_ratio']                                                            # free checked bag
# user['long_haul_ratio']                                                               # free checked bag

In [259]:
# user[cancellations]                                                               # no cancellation fees
user['cancel_rate'] = user['cancellations'].div(user['bookings'])                   # no cancellation fees
user['conversion_rate'] = user['bookings'].div(user['searches'])                    # no cancellation fees 
user['late_booking_ratio'] = user['late_booking'].div(user['trips'])                # no cancellation fees

In [260]:
user['slippage_rate'] = user['non_booking_sessions'].div(user['searches'])                                                  # exclusive discounts
user['avg_comparable_trip_discount'] = group.apply(lambda x: x[x['trip']==True]['comparable_trip_discount_ratio'].mean())   # exclusive discounts
user['discount_trip_ratio'] = user['discount_trip'].div(user['trips'])                                                      # exclusive discounts
user['clicks_per_session'] = (group['search_session_clicks'].sum()).div(user['searches'])                                   # exclusive discounts
user['browsing_mins_per_session'] = (group['search_session_mins'].sum()).div(user['searches'])                              # exclusive discounts
# user['avg_days_pre_travel']                                                                                               # exclusive discounts
# user['late_booking_ratio']                                                                                                # exclusive discounts

In [261]:
user['early_booking_ratio'] = user['early_booking'].div(user['trips'])                                    # backup for exclusive discounts                  
user['60_to_180_day_advance_booking_ratio'] = user['booking_60_to_180_days'].div(user['trips'])           # backup for exclusive discounts                  

In [262]:
user.reset_index(inplace=True) # reset the index to make user_id a column

In [263]:
user_col_drop = [
    'combo_trip',
    'foreign_trip',
    'discount_trip',
    'quality_travel',
    'long_haul_trip',
    'budget_flight',
    'premium_flight',
    'extended_return_trip',
    'staycation',
    'long_stay',
    'early_booking',
    'late_booking',
    'booking_60_to_180_days'
]
user = user.drop(columns=user_col_drop)         # remove unnecessary columns

In [264]:
# tidy up
convert_dtype(user, list(user.columns), dtype='float')
convert_dtype(user, ['user_id'], dtype='object')
user.fillna(0, inplace=True)                        # replace NaN with 0
user.replace([np.inf, -np.inf], 0, inplace=True)    # replace inf and -inf with 0

In [265]:
user.to_csv(CONFIG_PATH["user"], index=False)

#### 2. Correlation and sample size analyses => corr_matrix df & sample_size_matrix df
* Pearson correlation: Strong (|r| ≥ 0.7), Moderate (0.3≤ ∣r∣ <0.70), Weak (∣r∣ < 0.3)

In [266]:
user = pd.read_csv(CONFIG_PATH["user"])                        # import data df if available
convert_dtype(user, list(user.columns), dtype='float')
convert_dtype(user, ['user_id'], dtype='object')
df_name_and_variable_sizes(user, ['user_id'])     # print counts of unique user_id and unique session_id
# df name is user ==> 5998 user_id

df name is user ==> 5998 user_id


In [267]:
# preferred perk customer index
ppc = user[['monetization_rate', 'trips', 
            'days_cut_off_vs_sign_up', 
            'travellers_per_trip', 
              'tcv_per_traveller', 
              'tcv_per_room_unit', 
              'avg_tcv_per_seat_per_km']]
ppc_corr_matrix = ppc.corr(numeric_only=True)
ppc_corr_matrix.to_csv(r"C:\\data\\tt\\py\\ppc_corr_matrix.csv", index=True)

In [268]:
# free hotel meal index
fhm = user[['hotel_guests_per_trip', 
            'quality_travel_ratio', 
            'long_stay_ratio', 
            'staycation_ratio', 
            'foreign_trip_ratio', 
            'long_haul_ratio']]
fhm_corr_matrix = fhm.corr(numeric_only=True)
fhm_corr_matrix.to_csv(r"C:\\data\\tt\\py\\fhm_corr_matrix.csv", index=True)

In [269]:
# free hotel night index
fhn = user[['seats_per_trip', 
            'flight_only_ratio', 
            'premium_flight_ratio', 
              'extended_return_trip_ratio', 
              'foreign_trip_ratio', 
              'long_haul_ratio']]
fhn_corr_matrix = fhn.corr(numeric_only=True)
fhn_corr_matrix.to_csv(r"C:\\data\\tt\\py\\fhn_corr_matrix.csv", index=True)

In [270]:
# free checked bag index
fcb = user[['seats_per_trip', 
            'max_bags_per_seat', 
            'budget_flight_ratio', 
            'extended_return_trip_ratio', 
            'foreign_trip_ratio', 
            'long_haul_ratio']]
fcb_corr_matrix = fcb.corr(numeric_only=True)
fcb_corr_matrix.to_csv(r"C:\\data\\tt\\py\\fcb_corr_matrix.csv", index=True)

In [271]:
# no cancellation fees index
ncf = user[['conversion_rate', 
    'cancellations', 'cancel_rate', 
    'late_booking_ratio']]
ncf_corr_matrix = ncf.corr(numeric_only=True)
ncf_corr_matrix.to_csv(r"C:\\data\\tt\\py\\ncf_corr_matrix.csv", index=True)

In [272]:
# exclusive discounts index
edc = user[['clicks_per_session', 'browsing_mins_per_session', 
            'slippage_rate', 
            'avg_comparable_trip_discount', 'discount_trip_ratio', 
            'avg_days_pre_travel',
              'late_booking_ratio']]
edc_corr_matrix = edc.corr(numeric_only=True)
edc_corr_matrix.to_csv(r"C:\\data\\tt\\py\\edc_corr_matrix.csv", index=True)

In [273]:
# Calculate Pearson correlation between all possible pairs of columns
corr_matrix = user.corr(numeric_only=True)
# Export for future use
corr_matrix.to_csv(CONFIG_PATH["corr_matrix"], index=True)

In [277]:
# Generate matrix of number of unique user_ids within the number stdev of values in each column
sample_size_matrix = sample_size_matrix(user, reference_col='user_id', threshold=2)
# Export for future use
sample_size_matrix.to_csv(CONFIG_PATH["sample_size_matrix"], index=False)

# G. Scale sub-metrics, calculate perk-based behavioral metric and run K-Means
* Generate new df with perk allocation and K-Means clusters (selection df)

#### 0. Express lane: Import user df if available

In [20]:
user = pd.read_csv(CONFIG_PATH["user"])                        # import data df if available
convert_dtype(user, list(user.columns), dtype='float')
convert_dtype(user, ['user_id'], dtype='object')
df_name_and_variable_sizes(user, ['user_id'])     # print counts of unique user_id and unique session_id
# df name is user ==> 5998 user_id

df name is user ==> 5998 user_id


#### 1. Set up perk dictionary

In [21]:
perk_dict = {
    'PPC_Index': {
        "Valued customer": {
            'monetization_rate': 0.083333333,
            'trips': 0.083333333,
            'days_cut_off_vs_sign_up': 0.166666667,
            'travellers_per_trip': 0.166666667,
            'tcv_per_traveller': 0.166666667,
            'tcv_per_room_unit': 0.166666667,
            'avg_tcv_per_seat_per_km': 0.166666667
        }
    },
    'FHM_Index': {
        "Free hotel meal": {
            'hotel_guests_per_trip': 0.16666666666666666,
            'quality_travel_ratio': 0.16666666666666666,
            'long_stay_ratio': 0.16666666666666666,
            'staycation_ratio': 0.16666666666666666,
            'foreign_trip_ratio': 0.16666666666666666,
            'long_haul_ratio': 0.16666666666666666
        }
    },
    'FHN_Index': {
        "Free hotel night": {
            'seats_per_trip': 0.16666666666666666,
            'flight_only_ratio': 0.16666666666666666,
            'premium_flight_ratio': 0.16666666666666666,
            'extended_return_trip_ratio': 0.16666666666666666,
            'foreign_trip_ratio': 0.16666666666666666,
            'long_haul_ratio': 0.16666666666666666
        }
    },
    'FCB_Index': {
        "Free checked bag": {
            'seats_per_trip': 0.16666666666666666,
            'max_bags_per_seat': 0.16666666666666666,
            'budget_flight_ratio': 0.16666666666666666,
            'extended_return_trip_ratio': 0.16666666666666666,
            'foreign_trip_ratio': 0.16666666666666666,
            'long_haul_ratio': 0.16666666666666666
        }
    },
    'NCF_Index': {
        "No cancellation fees": {
            'cancellations': 0.166666666666667,
            'cancel_rate': 0.166666666666667,
            'conversion_rate': 0.333333333333333,
            'late_booking_ratio': 0.333333333333333
        }
    },
    'EDC_Index': {
        "Exclusive discounts": {
            'clicks_per_session': 0.083333333,
            'browsing_mins_per_session': 0.083333333,
            'slippage_rate': 0.166666667,
            'avg_comparable_trip_discount': 0.166666667,
            'discount_trip_ratio': 0.166666667,
            'avg_days_pre_travel': 0.166666667,
            'late_booking_ratio': 0.166666667, 
        }
    }
}

In [22]:
# convert perk_dict to perk df
flat_list = []
# Iterate through the nested dictionary and flatten it
for index, metrics in perk_dict.items():
    for desc, metric_weight_dict in metrics.items():
        for metric, weight in metric_weight_dict.items():
            flat_list.append([index, desc, metric, weight])
# Create the DataFrame
perk = pd.DataFrame(flat_list, columns=['index', 'perk', 'metric', 'weight'])
name = next((x for x in globals() if globals()[x] is perk), None)
perk.to_csv(f"C:\\data\\tt\\py\\{name}.csv", index=False)

In [23]:
perk_cluster = perk[['index', 'perk']].drop_duplicates(keep='first').reset_index(drop=True)
perk_cluster.to_csv(r"C:\\data\\tt\\py\\perk_cluster.csv", index=False) 

In [24]:
preselection = user.copy() # initialize df to add index scores, use until the penultimate step

#### 2. Boost high value customers for premium perks - PPC/VIP Index: Preferred perk customer index
* Valued customers are cut off at 80th-percentile or higher in terms of their consolidated PPC score on monetization rates, seasoning on platform, bulk booking, unit value sold and spend $ related metrics
* Favor valued customers by giving bonus points to boost their scores for exquisite perks i.e. free hotel meal or free hotel night
* Boost up to an extra 0.50 points to the raw premium perk scores of valued customers, prior to final renormalization by min-max scaling
* Composition metrics: 'monetization_rate', 'trips', 'days_cut_off_vs_sign_up', 'travellers_per_trip', 'tcv_per_traveller', 'tcv_per_room_unit', 'avg_tcv_per_seat_per_km'

In [28]:
choice_ppc = list(perk_dict.keys())[0]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_ppc, # edit here
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)
output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_ppc}.csv", index=False)
preselection[choice_ppc] = output[choice_ppc]

PPC_Index: ['monetization_rate', 'trips', 'days_cut_off_vs_sign_up', 'travellers_per_trip', 'tcv_per_traveller', 'tcv_per_room_unit', 'avg_tcv_per_seat_per_km']
Scaling column monetization_rate:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.67. Mean=0.30, Std=0.19
- Scaled: Final Min=0.00, Max=1.00. Mean=0.44, Std=0.27

Scaling column trips:
- Initial: Actual Min=0.00, Max=8.00. Capped Min=0.00, Max=5.33. Mean=2.38, Std=1.47
- Scaled: Final Min=0.00, Max=1.00. Mean=0.44, Std=0.27

Scaling column days_cut_off_vs_sign_up:
- Initial: Actual Min=72.00, Max=737.00. Capped Min=119.18, Max=260.62. Mean=189.90, Std=35.36
- Scaled: Final Min=0.00, Max=1.00. Mean=0.49, Std=0.18

Scaling column travellers_per_trip:
- Initial: Actual Min=0.00, Max=3.00. Capped Min=0.15, Max=1.90. Mean=1.03, Std=0.44
- Scaled: Final Min=0.00, Max=1.00. Mean=0.50, Std=0.22

Scaling column tcv_per_traveller:
- Initial: Actual Min=0.00, Max=7065.36. Capped Min=0.00, Max=2021.80. Mean=792.55, Std=614.63


In [26]:
# calculate bonus point for high value customers for their premium perk scores 
pp_threshold = preselection[choice_ppc].quantile(0.80) # impose 80% percentile threshold, i.e. top 20% users by PPC Index score
max_bonus_pt = 0.50
preselection['bonus_pt'] = np.where(preselection[choice_ppc] >= pp_threshold,
                                 np.maximum(((preselection[choice_ppc] - pp_threshold).div((1 - pp_threshold))) * max_bonus_pt, (max_bonus_pt * 0.5)),
                                 0) 

#### 3. Exquisite perks
* Top PPC scorers eligible for boost to scores
* Aim to entrench existing behavior primarily 

* FHM Index: Free hotel meal index
* Incentive for customers who bulk book for quality travel experience (premium airline or luxury hotel). Favor staycationers as well as those on long haul and extended overseas trips
* Composition metrics: 'hotel_guests_per_trip', 'quality_travel_ratio', 'long_stay_ratio', 'staycation_ratio', 'foreign_trip_ratio', 'long_haul_ratio'

In [29]:
choice_fhm = list(perk_dict.keys())[1]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_fhm, # edit here 
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)

output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_fhm}.csv", index=False)
preselection[choice_fhm] = (output[choice_fhm] + preselection['bonus_pt']).clip(upper=1) # use clip to cap value at 1
preselection[choice_fhm+'_original'] = output[choice_fhm]

FHM_Index: ['hotel_guests_per_trip', 'quality_travel_ratio', 'long_stay_ratio', 'staycation_ratio', 'foreign_trip_ratio', 'long_haul_ratio']
Scaling column hotel_guests_per_trip:
- Initial: Actual Min=0.00, Max=4.00. Capped Min=0.07, Max=1.94. Mean=1.00, Std=0.47
- Scaled: Final Min=0.00, Max=1.00. Mean=0.50, Std=0.23

Scaling column quality_travel_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.01, Max=1.00. Mean=0.74, Std=0.36
- Scaled: Final Min=0.00, Max=1.00. Mean=0.74, Std=0.36

Scaling column long_stay_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.61. Mean=0.11, Std=0.25
- Scaled: Final Min=0.00, Max=1.00. Mean=0.16, Std=0.31

Scaling column staycation_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.13. Mean=0.01, Std=0.06
- Scaled: Final Min=0.00, Max=1.00. Mean=0.01, Std=0.11

Scaling column foreign_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.13. Mean=0.01, Std=0.06
- Scaled: Final Min=0.00, Max=1

* FHN Index: One night free hotel with flight index
* Incentive for customers who bulk book but more more exclusively just flights, with a bias for premium airline on long haul and extended overseas trips
* Unexpectedness of reward may encourage future bookings with hotel as a plus to existing behavior  
* Composition metrics: 'seats_per_trip', 'flight_only_ratio', 'premium_flight_ratio', 'extended_return_trip_ratio', 'foreign_trip_ratio', 'long_haul_ratio'

In [30]:
choice_fhn = list(perk_dict.keys())[2]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_fhn, # edit here 
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)

output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_fhn}.csv", index=False)
preselection[choice_fhn] = (output[choice_fhn] + preselection['bonus_pt']).clip(upper=1)
preselection[choice_fhn+'_original'] = output[choice_fhn]

FHN_Index: ['seats_per_trip', 'flight_only_ratio', 'premium_flight_ratio', 'extended_return_trip_ratio', 'foreign_trip_ratio', 'long_haul_ratio']
Scaling column seats_per_trip:
- Initial: Actual Min=0.00, Max=3.00. Capped Min=0.00, Max=1.86. Mean=0.91, Std=0.47
- Scaled: Final Min=0.00, Max=1.00. Mean=0.49, Std=0.25

Scaling column flight_only_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.50. Mean=0.09, Std=0.20
- Scaled: Final Min=0.00, Max=1.00. Mean=0.16, Std=0.32

Scaling column premium_flight_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=1.00. Mean=0.57, Std=0.41
- Scaled: Final Min=0.00, Max=1.00. Mean=0.57, Std=0.41

Scaling column extended_return_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.51. Mean=0.09, Std=0.21
- Scaled: Final Min=0.00, Max=1.00. Mean=0.14, Std=0.31

Scaling column foreign_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.13. Mean=0.01, Std=0.06
- Scaled: Final Min=0

#### 4. Standard perks
* Favor less valued customers to incentivize better conversions for monetization (i.e. non-cancelled trips) by offering more flexibility with no cancellation fees or economic benefits like free checked bag and exclusive discounts
* Cheaper as do not reduce potential units sold
* Market efficiency by moving blocks of otherwise unsold and expiring units for perks with 'late_booking_ratio' metric component

* FCB Index: Free checked bag index
* Incentive for customers who bulk book flights for long haul and extended overseas trips which drive need for more baggage allowance, with a bias for budget airline or those with more than 1 checked bags per seat
* Composition metrics: 'seats_per_trip', 'max_bags_per_seat', 'budget_flight_ratio', 'extended_return_trip_ratio', 'foreign_trip_ratio', 'long_haul_ratio'

In [31]:
choice_fcb = list(perk_dict.keys())[3]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_fcb, # edit here 
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)

output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_fcb}.csv", index=False)
preselection[choice_fcb] = output[choice_fcb]

FCB_Index: ['seats_per_trip', 'max_bags_per_seat', 'budget_flight_ratio', 'extended_return_trip_ratio', 'foreign_trip_ratio', 'long_haul_ratio']
Scaling column seats_per_trip:
- Initial: Actual Min=0.00, Max=3.00. Capped Min=0.00, Max=1.86. Mean=0.91, Std=0.47
- Scaled: Final Min=0.00, Max=1.00. Mean=0.49, Std=0.25

Scaling column max_bags_per_seat:
- Initial: Actual Min=0.00, Max=5.00. Capped Min=0.00, Max=1.92. Mean=0.69, Std=0.61
- Scaled: Final Min=0.00, Max=1.00. Mean=0.35, Std=0.30

Scaling column budget_flight_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.91. Mean=0.25, Std=0.33
- Scaled: Final Min=0.00, Max=1.00. Mean=0.26, Std=0.34

Scaling column extended_return_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.51. Mean=0.09, Std=0.21
- Scaled: Final Min=0.00, Max=1.00. Mean=0.14, Std=0.31

Scaling column foreign_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.13. Mean=0.01, Std=0.06
- Scaled: Final Min=0.0

* NCF/FTC Index: No cancellation fees index (now Free to cancel)
* Incentive for customers with higher conversion rate from session to booking but reduced monetized trips due to cancellations, and those who tend to book late (e.g. less than a week prior to travel)
* May attact finnicky customers or business travellers who may be less price sensitive to purchase unsold units that risk expiring worthless (inventory obselence)
* Composition metrics: 'cancellations', 'cancel_rate', 'conversion_rate', 'late_booking_ratio'

In [32]:
choice_ncf = list(perk_dict.keys())[4]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_ncf, # edit here 
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)

output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_ncf}.csv", index=False)
preselection[choice_ncf] = output[choice_ncf]

NCF_Index: ['cancellations', 'cancel_rate', 'conversion_rate', 'late_booking_ratio']
Scaling column cancellations:
- Initial: Actual Min=0.00, Max=2.00. Capped Min=0.00, Max=0.71. Mean=0.10, Std=0.31
- Scaled: Final Min=0.00, Max=1.00. Mean=0.10, Std=0.29

Scaling column cancel_rate:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.37. Mean=0.04, Std=0.16
- Scaled: Final Min=0.00, Max=1.00. Mean=0.08, Std=0.26

Scaling column conversion_rate:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=0.68. Mean=0.31, Std=0.18
- Scaled: Final Min=0.00, Max=1.00. Mean=0.46, Std=0.26

Scaling column late_booking_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=1.00. Mean=0.42, Std=0.36
- Scaled: Final Min=0.00, Max=1.00. Mean=0.42, Std=0.36



* EDC/SOT Index: Exclusive discounts index (now Special offer travel)
* Incentive for customers who exhibit bargain hunting behavior such spending extended time clicking and browsing but with high slippage rate (i.e. searches that fail to lead to bookings) and evidence of meticulous frugality such as booking as many days in advance and at prices below the average (for similar travel location and travel period)
* Customers who tend to book late (within 7 days or less to date of travel) but otherwise show bargain hunting behavior also favored as means to shift units that risk expiring worthless with exclusive discounts
* Composition metrics: 'clicks_per_session', 'browsing_mins_per_session', 'slippage_rate', 'avg_comparable_trip_discount', 'discount_trip_ratio', 'avg_days_pre_travel', 'late_booking_ratio'

In [33]:
choice_edc = list(perk_dict.keys())[5]
# min-max scale relevant metrics and compute raw score
output = compute_index_scores(df=preselection, index_weight_dict=perk_dict, 
                                 selected_index=choice_edc, # edit here 
                                 col_filter=None, debug=True, 
                                 cap=True, threshold=2)

output['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in output if col != 'user_id']
output = output[cols]
output.to_csv(f"C:\\data\\tt\\py\\{choice_edc}.csv", index=False)
preselection[choice_edc] = output[choice_edc]

EDC_Index: ['clicks_per_session', 'browsing_mins_per_session', 'slippage_rate', 'avg_comparable_trip_discount', 'discount_trip_ratio', 'avg_days_pre_travel', 'late_booking_ratio']
Scaling column clicks_per_session:
- Initial: Actual Min=4.12, Max=84.12. Capped Min=4.79, Max=26.71. Mean=15.75, Std=5.48
- Scaled: Final Min=0.00, Max=1.00. Mean=0.49, Std=0.22

Scaling column browsing_mins_per_session:
- Initial: Actual Min=0.53, Max=10.42. Capped Min=0.60, Max=3.31. Mean=1.95, Std=0.68
- Scaled: Final Min=0.00, Max=1.00. Mean=0.49, Std=0.22

Scaling column slippage_rate:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.32, Max=1.00. Mean=0.69, Std=0.18
- Scaled: Final Min=0.00, Max=1.00. Mean=0.54, Std=0.26

Scaling column avg_comparable_trip_discount:
- Initial: Actual Min=-9.88, Max=0.99. Capped Min=-1.17, Max=0.99. Mean=0.10, Std=0.64
- Scaled: Final Min=0.00, Max=1.00. Mean=0.61, Std=0.21

Scaling column discount_trip_ratio:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max

#### 5. MinMax Index level scores

In [35]:
# run final minmax scaling on perk scores (stored in preselection df) and generate a dict and its df
list_to_minmax = [choice_fhm, choice_fhn, choice_fcb, choice_ncf, choice_edc] # list of index values
prelim_scores = preselection[list_to_minmax] # df for minmax
final_scaled_dict = minmax(prelim_scores, col_list=list_to_minmax, 
                           col_filter=None, debug=True, 
                           cap=False, threshold=None) # no need to cap the min and max at 2 stdev
final_scaled = pd.DataFrame(final_scaled_dict)                          # to use for finalization later
final_scaled['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in final_scaled if col != 'user_id']
final_scaled = final_scaled[cols]
final_scaled.to_csv(f"C:\\data\\tt\\py\\final_scaled.csv", index=False) # export for future use

Scaling column FHM_Index:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=1.00. Mean=0.30, Std=0.18
- Scaled: Final Min=0.00, Max=1.00. Mean=0.30, Std=0.18

Scaling column FHN_Index:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=1.00. Mean=0.29, Std=0.21
- Scaled: Final Min=0.00, Max=1.00. Mean=0.29, Std=0.21

Scaling column FCB_Index:
- Initial: Actual Min=0.00, Max=0.88. Capped Min=0.00, Max=0.88. Mean=0.22, Std=0.15
- Scaled: Final Min=0.00, Max=1.00. Mean=0.25, Std=0.17

Scaling column NCF_Index:
- Initial: Actual Min=0.00, Max=0.94. Capped Min=0.00, Max=0.94. Mean=0.32, Std=0.19
- Scaled: Final Min=0.00, Max=1.00. Mean=0.34, Std=0.20

Scaling column EDC_Index:
- Initial: Actual Min=0.17, Max=0.78. Capped Min=0.17, Max=0.78. Mean=0.41, Std=0.10
- Scaled: Final Min=0.00, Max=1.00. Mean=0.40, Std=0.16



In [34]:
# run minmax on the original scores stored in preselection df and generate a dict
orig_to_minmax = [choice_fhm+'_original', choice_fhn+'_original'] # list of index values
orig_prelim_scores = preselection[orig_to_minmax] # df for minmax
orig_scaled_dict = minmax(orig_prelim_scores, col_list=orig_to_minmax, 
                           col_filter=None, debug=True, 
                           cap=False, threshold=None) # no need to cap the min and max at 2 stdev

orig_scaled = pd.DataFrame(orig_scaled_dict)                          # to use for finalization later
orig_scaled['user_id'] = preselection['user_id']
cols = ['user_id'] + [col for col in orig_scaled if col != 'user_id']
orig_scaled = orig_scaled[cols]
orig_scaled.to_csv(f"C:\\data\\tt\\py\\orig_scaled.csv", index=False) # export for future use

Scaling column FHM_Index_original:
- Initial: Actual Min=0.00, Max=0.96. Capped Min=0.00, Max=0.96. Mean=0.25, Std=0.13
- Scaled: Final Min=0.00, Max=1.00. Mean=0.26, Std=0.13

Scaling column FHN_Index_original:
- Initial: Actual Min=0.00, Max=1.00. Capped Min=0.00, Max=1.00. Mean=0.24, Std=0.16
- Scaled: Final Min=0.00, Max=1.00. Mean=0.24, Std=0.16



#### 6. K-Means

* K-Means on transaction level data

In [36]:
clean = pd.read_csv(CONFIG_PATH["clean"])                        # import data df if available
clean_dtype_dict = data_dtype_dict
convert_dtype_multi(clean, dtype_dict=clean_dtype_dict)          # clean and data df have same columns hence same columns need prepping for correct dtype
df_name_and_variable_sizes(clean, ['user_id', 'session_id'])     # print counts of unique user_id and unique session_id
# df name is clean ==> 5998 user_id , 47181 session_id

C:\Users\Master\AppData\Local\Temp\ipykernel_43012\3397598374.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21,22,23,24,26,27,29,30,31,32,33,34,35,36,37,38,39,40,41,43,44,49,50,51,52,53,54,55,56,57,59,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  clean = pd.read_csv(CONFIG_PATH["clean"])                        # import data df if available


df name is clean ==> 5998 user_id , 47980 session_id


In [37]:
kmeans_t_var = [
    'user_id', 'birthdate', 'days_cut_off_vs_sign_up', 'home_airport_lat', 'home_airport_lon', 
    'sign_up_date', 'session_start', 'session_end', 'flight_discount_amount',
    'hotel_discount_amount', 'page_clicks', 'seats', 'departure_time', 
    'return_time', 'checked_bags', 'destination_airport_lat', 'destination_airport_lon', 
    'base_fare_usd', 'nights', 'rooms', 'check_in_time', 'check_out_time', 
    'hotel_per_room_usd', 'use_trip_id', 'distance_km', 'booking', 'cancelled_booking', 
    'trip', 'trip_with_flight', 'trip_with_hotel', 'combo_trip', 'flight_only_trip', 
    'hotel_only_trip', 'return_trip', 'foreign_trip', 'long_haul_trip', 'staycation', 
    'quality_travel', 'discount_trip', 'session_duration_mins', 'travel_start', 
    'travel_end', 'travel_days', 'trip_booking_time', 'days_pre_travel', 'long_stay', 
    'long_travel', 'early_booking', 'late_booking', 'booking_60_to_180_days', 
    'num_travellers', 'bags_per_seat', 
    'guests_per_room', 'base_fare_usd_per_km', 'room_nights', 'value_per_traveller', 
    'value_per_seat_per_km', 'comparable_trip_discount_ratio'
]

In [38]:
kmeans_t = clean[kmeans_t_var].copy()                                  # initialize relevant kmeans df
kmeans_t = user_cluster(kmeans_t, groupby_column='user_id',            # add kmeans cluster to the df
                      n_clusters=5, random_state=42, na_as_zero=True, 
                      numeric_only=True, n_init=10)
kmeans_t = kmeans_t.groupby('user_id')['cluster'].first().reset_index()     # reset index to be exported as table of user_id and corresponding cluster
kmeans_t = kmeans_t.drop_duplicates(keep='first')                           # drop potential duplicated rows except first occurrence
kmeans_t.rename(columns={'cluster': 'kmeans_t'}, inplace=True)              # rename col
kmeans_t['kmeans_t'] = "T" + (kmeans_t['kmeans_t'] + 1).astype(str)         # rename values
kmeans_t.to_csv(CONFIG_PATH["kmeans_t"], index=False)                       # export for future use

* K-Means on user metrics

In [39]:
user = pd.read_csv(CONFIG_PATH["user"])                         # import data df if available
convert_dtype(user, list(user.columns), dtype='float')
convert_dtype(user, ['user_id'], dtype='object')
df_name_and_variable_sizes(user, ['user_id'])                   # print counts of unique user_id and unique session_id
# df name is user ==> 5998 user_id

df name is user ==> 5998 user_id


In [77]:
metrics_list = [
    "user_id",
    "avg_comparable_trip_discount",
    "avg_days_pre_travel",
    "avg_tcv_per_seat_per_km",
    "browsing_mins_per_session",
    "budget_flight_ratio",
    "cancel_rate",
    "cancellations",
    "clicks_per_session",
    "conversion_rate",
    "days_cut_off_vs_sign_up",
    "discount_trip_ratio",
    "extended_return_trip_ratio",
    "flight_only_ratio",
    "foreign_trip_ratio",
    "hotel_guests_per_trip",
    "late_booking_ratio",
    "long_haul_ratio",
    "long_stay_ratio",
    "max_bags_per_seat",
    "monetization_rate",
    "premium_flight_ratio",
    "quality_travel_ratio",
    "seats_per_trip",
    "slippage_rate",
    "staycation_ratio",
    "tcv_per_room_unit",
    "tcv_per_traveller",
    "travellers_per_trip",
    "trips"
]

In [78]:
kmeans_u = user[metrics_list]                                           # initialize relevant kmeans df
kmeans_u = user_cluster(kmeans_u, groupby_column='user_id',             # add kmeans cluster to the df
                      n_clusters=5, random_state=42, na_as_zero=True, 
                      numeric_only=True, n_init=10)
kmeans_u = kmeans_u.groupby('user_id')['cluster'].first().reset_index() # reset index to be exported as table of user_id and corresponding cluster
kmeans_u = kmeans_u.drop_duplicates(keep='first')                       # drop potential duplicated rows except first occurrence
kmeans_u.rename(columns={'cluster': 'kmeans_u'}, inplace=True)          # rename col
kmeans_u['kmeans_u'] = "U" + (kmeans_u['kmeans_u'] + 1).astype(str)         # rename values
kmeans_u.to_csv(CONFIG_PATH["kmeans_u"], index=False)                   # export for future use

C:\Users\Master\AppData\Local\Temp\ipykernel_43012\2652901594.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.fillna(0, inplace=True)


#### 7. Finalize selection df to allocate perks and add in kmeans clustering  

In [68]:
# concatenate user and user derived dfs which have the same number and order of user_id
selection = pd.DataFrame(pd.concat([user, preselection[[choice_ppc, 'bonus_pt']], orig_scaled[orig_to_minmax], final_scaled[list_to_minmax]], axis=1)) # initialize selection df
cols_to_rank_check = list_to_minmax # keep same order of cols as final_scaled

# calculate ranks for each col and store them as new columns with rank post-fix
for col in cols_to_rank_check:
    new_col_name = f"{col}_rank"
    selection[new_col_name] = selection[col].rank(ascending=False)

In [69]:
# Allocate based on rank first and if fail then by score 

# list of rank cols to use to allocate
rank_cols_to_allocate = ['FHM_Index_rank', 'FCB_Index_rank', 'FHN_Index_rank', 'NCF_Index_rank', 'EDC_Index_rank']

# Primary allocation by rank
selection['primary_allocation_index'] = selection[rank_cols_to_allocate].apply(
    lambda row: row.idxmin().replace('_rank', '') if sum(row == row.min()) == 1 else "allocate by score",
    axis=1
)

# Secondary allocation by score (if unallocated by rank)
selection['final_allocation_index'] = selection['primary_allocation_index']
mask = selection['primary_allocation_index'] == "allocate by score"
selection.loc[mask, 'final_allocation_index'] = selection.loc[mask, cols_to_rank_check].idxmax(axis=1)

In [ ]:
# Alternative - allocate based on score first and if fail then by rank

# list of rank cols to use to allocate
rank_cols_to_allocate = ['FHM_Index_rank', 'FCB_Index_rank', 'FHN_Index_rank', 'NCF_Index_rank', 'EDC_Index_rank']

# primary allocation
selection['primary_allocation_index'] = np.where(selection[cols_to_rank_check].apply(lambda x: sum(x == x.max()), axis=1)==1,   # number of cols with top score==1
                                        selection[cols_to_rank_check].apply(lambda row: row.idxmax(), axis=1), 
                                        "allocate by rank")                                                                     # initial allocation
# secondary allocation
selection['final_allocation_index'] = np.where(
    selection['primary_allocation_index'] == "allocate by rank",
    selection[rank_cols_to_allocate].apply(lambda row: row.idxmin().replace('_rank', ''), axis=1),
    selection['primary_allocation_index']) # final allocation



In [71]:
# remove primary allocation col and simplify col name for the final allocation
selection['index'] = selection['final_allocation_index']
selection.drop(columns=['primary_allocation_index', 'final_allocation_index'], axis=1, inplace=True)           # drop useless col

In [72]:
# perk
selection = selection.merge(
    perk_cluster[['index', 'perk']],
    on='index', how='left')

In [73]:
# kmeans cluster names to user_id
selection = selection.merge(
    kmeans_t[['user_id', 'kmeans_t']],
    on='user_id', how='left')

selection = selection.merge(
    kmeans_u[['user_id', 'kmeans_u']],
    on='user_id', how='left')

In [74]:
selection.to_csv(CONFIG_PATH["selection"], index=False)           # export for future use

#### 8. Set analysis

In [75]:
# perk vs kmeans_t
seed_1 = selection.groupby(['perk', 'kmeans_t'])['user_id'].nunique().reset_index()       # initialize a groupby
matrix_1 = seed_1.pivot(index='kmeans_t', columns='perk', values='user_id').fillna(0).astype(int)
custom_order = ['free_hotel_meal', 'free_checked_bag', 'free_hotel_night', 
                'no_cancellation_fees', 'exclusive_discounts']      # to reorder perk cols
matrix_1 = matrix_1.reindex(columns=custom_order)                   # reorder cols
matrix_1.to_csv(f"C:\\data\\tt\\py\\set_1.csv", index=True)

# perk vs kmeans_u
seed_2 = selection.groupby(['perk', 'kmeans_u'])['user_id'].nunique().reset_index()       # initialize a groupby
matrix_2 = seed_2.pivot(index='kmeans_u', columns='perk', values='user_id').fillna(0).astype(int)
matrix_2 = matrix_2.reindex(columns=custom_order)                   # reorder cols
matrix_2.to_csv(f"C:\\data\\tt\\py\\set_2.csv", index=True)

# kmeans_u vs kmeans_t
seed_3 = selection.groupby(['kmeans_u', 'kmeans_t'])['user_id'].nunique().reset_index()           # initialize a groupby
matrix_3 = seed_3.pivot(index='kmeans_t', columns='kmeans_u', values='user_id').fillna(0).astype(int)
matrix_3.to_csv(f"C:\\data\\tt\\py\\set_3.csv", index=True)